# **다중 회귀분석 (Multiple linear regression analysis)**
- 연역적 접근 : 숲을 보고 나무를 파악 (통계학적 추론)

* 가설설정 (분석결과)
- 지하철 거리km가 가까울 수록 공급가액이 비쌀 것이다. -> 거리가 멀어질 수록 음의 상관관계가 있을 것 (음의 상관관계)
- 장례식장과 같은 NIMBY 시설의 여부는 공급가액을 떨어트릴 것 -> 주변에 장례식장이 있다면 음의 상관관계가 있을 것 (음의 상관관계)
- 최고층은 높을 수록 비쌀 것 (양의 상관관계)
- 전용면적, 공급면적이 넓을 수록 비쌀 것 (양의 상관관계)
- 대형건설사가 포함되어 있다면 가격이 비싸길 것 (양의 상관관계)
- 세대 수가 많을 수록 가격이 비싸지 않을까?
- 주변에 병원, 보건의료기관이 있다면 비싸질 것
- 주변에 대학교가 있다면?
- 용적률과 건폐율이 높을 수록

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

# **(ver6) CSV 파일 전처리 코드**
- 아래 전처리 진행 후 ver7으로 저장

### **1) 화성 봉담2지구 '전용율 NaN'**
- 네이버, 호갱 노노 검색 후 처리

In [246]:
# 전용율 NaN
# 화성_봉담2지구 = org_apt_data.loc[org_apt_data['아파트명'] == '화성 봉담2지구 B-1BL 중흥S-클래스']
# 화성_봉담2지구[['공급면적', '전용율']]

,공급면적,전용율
1962,108.0507,NaN
1963,109.4360,NaN
1964,108.4359,NaN


In [247]:
# 전용율 : (전용면적 / 공급면적) * 100

# org_apt_data.loc[(org_apt_data['아파트명'] == '화성 봉담2지구 B-1BL 중흥S-클래스') & (org_apt_data['공급면적'] == 108.0507), '전용율'] = 78
# org_apt_data.loc[(org_apt_data['아파트명'] == '화성 봉담2지구 B-1BL 중흥S-클래스') & (org_apt_data['공급면적'] == 109.4360), '전용율'] = 77
# org_apt_data.loc[(org_apt_data['아파트명'] == '화성 봉담2지구 B-1BL 중흥S-클래스') & (org_apt_data['공급면적'] == 108.4359), '전용율'] = 78

In [248]:
# 전용율_바꾼것 = org_apt_data.loc[org_apt_data['아파트명'] == '화성 봉담2지구 B-1BL 중흥S-클래스']
# 전용율_바꾼것[['공급면적', '전용율']]

,공급면적,전용율
1962,108.0507,78.0
1963,109.4360,77.0
1964,108.4359,78.0


In [250]:
# 전용율 채운 후 결측값 확인
# org_apt_data.isna().sum()

아파트명              0
법정동주소             0
위도                0
경도                0
세대수               0
임대세대수           234
최고층              36
최저층              45
최대공급면적            0
최소공급면적            0
총아파트동수           26
용적률            1313
건폐율            1376
세대평균_주차대수        70
공급면적              0
전용면적              0
전용율               0
방수               84
욕실수              84
현관구조              0
입주예정일             0
공급액(만원)           0
대형건설사             0
location          0
지하철역              0
지하철역_거리(km)       0
1차병원              0
2차병원              0
3차병원              0
공원                0
대학                0
역사명               0
노선명_리스트           0
구                 0
광역시도              0
소매                0
음식                0
교육                0
장례식장              0
보건의료              0
유원지오락             0
총인구수              0
dtype: int64

### **2) 양주회천 A17블록 '세대수' & '아파트명 변경**

In [240]:
# filtered_df = org_apt_data[org_apt_data['아파트명'] == '양주회천 A17블록 신혼희망타운(공공분양)'] 
# filtered_df['세대수'] 

803    506
804    506
805    506
806    506
807    506
808    506
809    506
810    506
Name: 세대수, dtype: int64

In [ ]:
# 기존 세대수 506 -> 변경할 세대수 760
# org_apt_data.loc[org_apt_data['아파트명'] == '양주회천 A17블록 신혼희망타운(공공분양)', '세대수'] = 760
# org_apt_data.loc[org_apt_data['아파트명'] == '양주회천 A17블록 신혼희망타운(공공분양)', '아파트명'] = '양주회천 센트레빌'

In [243]:
# filtered_df = org_apt_data[org_apt_data['아파트명'] == '양주회천 센트레빌'] 
# filtered_df['세대수'] # 기존 세대수 506 -> 변경할 세대수 760

803    760
804    760
805    760
806    760
807    760
808    760
809    760
810    760
Name: 세대수, dtype: int64

### **3) 아파트명 변경**

In [239]:
# # 수정전 수정 할 아파트명 확인 
# amended_result_list = []
# apt_names = org_apt_data['아파트명'].value_counts()

# for apt_name, count in apt_names.items() : 
#     amended_result_list.append(f'{apt_name} : {count}')
    
# amended_result_list

['고덕강일 제일풍경채 : 27',
 'e편한세상 강일 어반브릿지 : 26',
 '송도자이 크리스탈오션 : 22',
 '송도자이 더 스타 : 20',
 '수유동 북한산 스카이뷰 아파트 : 18',
 '힐스테이트 봉담 프라이드시티 : 17',
 '힐스테이트 몬테로이 2BL : 17',
 '장위자이 레디언트 : 16',
 '남구로역 동일 센타시아 : 16',
 '송도 Luxe Ocean SK VIEW : 16',
 '봉담자이 프라이드시티 : 16',
 '올림픽파크 포레온 : 16',
 '오포자이 디 오브 C-4BL : 16',
 '힐스테이트 리슈빌 강일 : 15',
 '운정자이 시그니처 : 15',
 '래미안 엘리니티 : 15',
 '오산 세교지구 Cd-1블록 호반써밋 라테라스 : 15',
 '힐스테이트 몬테로이 3BL : 14',
 '힐스테이트 몬테로이 1BL : 14',
 '북수원자이 렉스비아 : 14',
 '금곡역한신더휴 : 13',
 '운정 아이파크 더 테라스 : 13',
 '리버센 SK VIEW 롯데캐슬 : 13',
 '래미안 라그란데 : 13',
 '계양 서해그랑블 더테라스 : 12',
 '봉담자이 라피네 : 12',
 '오포자이 디 오브 C-3BL : 12',
 '북서울자이 폴라리스 : 12',
 '부평 SK VIEW 해모로 : 12',
 '화성향남 상신지구 A1-1블록 향남역 한양수자인 디에스티지 : 12',
 'DMC파인시티자이 : 12',
 '반정 아이파크 캐슬 5단지 : 12',
 '영종 오션파크 모아엘가 그랑데 : 12',
 '디에이치 퍼스티어 아이파크 : 11',
 '힐스테이트 레이크 송도 3차 : 11',
 '힐스테이트 초월역 1BL : 11',
 'DMC센트럴자이 : 11',
 '두산위브더제니스 센트럴 여의 : 11',
 '강동 밀레니얼 중흥S-클래스 : 11',
 '양우내안애 더 센트럴 : 11',
 '대치 푸르지오 써밋 : 11',
 '안양 어반포레 자연앤 e편한세상(국민) : 11',
 '인덕원 자이 SK VIEW : 11',
 '

In [241]:
# org_apt_data.loc[org_apt_data['아파트명'] == '안양 어반포레 자연앤 e편한세상(국민)', '아파트명'] = '안양 어반포레 자연앤 e편한세상'

# org_apt_data.loc[org_apt_data['아파트명'] == 'e편한세상 동탄 파크아너스 A56블럭(2회차)', '아파트명'] = 'e편한세상 동탄 파크아너스 A56블럭'
# org_apt_data.loc[org_apt_data['아파트명'] == '힐스테이트 검단 웰카운티(민영주택)', '아파트명'] = '힐스테이트 검단 웰카운티'
# org_apt_data.loc[org_apt_data['아파트명'] == 'SH 위례지구 A1-5BL 공공분양', '아파트명'] = 'SH 위례지구 A1-5BL'
# org_apt_data.loc[org_apt_data['아파트명'] == '송도 베르디움퍼스트(RC-4) 외국인임대 분양전환 아파트', '아파트명'] = '송도 베르디움퍼스트(RC-4)'
# org_apt_data.loc[org_apt_data['아파트명'] == '과천지식정보타운 린 파밀리에(공공분양)', '아파트명'] = '과천지식정보타운 린 파밀리에'
# org_apt_data.loc[org_apt_data['아파트명'] == '안양 어반포레 자연앤 e편한세상(민영)', '아파트명'] = '안양 어반포레 자연앤 e편한세상'
# org_apt_data.loc[org_apt_data['아파트명'] == 'SH 위례지구 A1-12BL 공공분양', '아파트명'] = 'SH 위례지구 A1-12BL'
# org_apt_data.loc[org_apt_data['아파트명'] == '더샵 송도 그린워크3차(D17-1블록) 외국인임대 분양전환 APT', '아파트명'] = '더샵 송도 그린워크3차(D17-1블록)'
# org_apt_data.loc[org_apt_data['아파트명'] == '송도 에듀포레푸르지오(RC-2) 외국인임대 분양전환 아파트', '아파트명'] = '송도 에듀포레푸르지오(RC-2)'
# # org_apt_data.loc[org_apt_data['아파트명'] == '장항지구 A-5블록 신혼희망타운', '아파트명'] = '장항지구 A-4블록 신혼희망타운'
# org_apt_data.loc[org_apt_data['아파트명'] == '인천 만석웰카운티 공공임대 분양전환 후 일반분양', '아파트명'] = '인천 만석웰카운티'
# org_apt_data.loc[org_apt_data['아파트명'] == '오산 세교2지구 A-4블록 중흥S-클래스 에듀파크', '아파트명'] = '오산 세교2지구 중흥S-클래스'

In [242]:
# # 수정한 내용 확인

# amended_result_list = []
# apt_names = org_apt_data['아파트명'].value_counts()

# for apt_name, count in apt_names.items() : 
#     amended_result_list.append(f'{apt_name} : {count}')

# amended_result_list

['고덕강일 제일풍경채 : 27',
 'e편한세상 강일 어반브릿지 : 26',
 '송도자이 크리스탈오션 : 22',
 '송도자이 더 스타 : 20',
 'e편한세상 동탄 파크아너스 A56블럭 : 18',
 '수유동 북한산 스카이뷰 아파트 : 18',
 '힐스테이트 봉담 프라이드시티 : 17',
 '힐스테이트 몬테로이 2BL : 17',
 '장위자이 레디언트 : 16',
 '오포자이 디 오브 C-4BL : 16',
 '송도 Luxe Ocean SK VIEW : 16',
 '봉담자이 프라이드시티 : 16',
 '남구로역 동일 센타시아 : 16',
 '올림픽파크 포레온 : 16',
 '래미안 엘리니티 : 15',
 '운정자이 시그니처 : 15',
 '오산 세교지구 Cd-1블록 호반써밋 라테라스 : 15',
 '힐스테이트 리슈빌 강일 : 15',
 '힐스테이트 몬테로이 3BL : 14',
 '북수원자이 렉스비아 : 14',
 '힐스테이트 몬테로이 1BL : 14',
 '안양 어반포레 자연앤 e편한세상 : 14',
 '리버센 SK VIEW 롯데캐슬 : 13',
 '래미안 라그란데 : 13',
 '금곡역한신더휴 : 13',
 '운정 아이파크 더 테라스 : 13',
 '북서울자이 폴라리스 : 12',
 'DMC파인시티자이 : 12',
 '봉담자이 라피네 : 12',
 '부평 SK VIEW 해모로 : 12',
 '계양 서해그랑블 더테라스 : 12',
 '반정 아이파크 캐슬 5단지 : 12',
 '오포자이 디 오브 C-3BL : 12',
 '영종 오션파크 모아엘가 그랑데 : 12',
 '화성향남 상신지구 A1-1블록 향남역 한양수자인 디에스티지 : 12',
 '강동 밀레니얼 중흥S-클래스 : 11',
 '두산위브더제니스 센트럴 여의 : 11',
 '롯데캐슬 이스트폴 : 11',
 '디에이치 퍼스티어 아이파크 : 11',
 '대치 푸르지오 써밋 : 11',
 '힐스테이트 초월역 1BL : 11',
 '양우내안애 더 센트럴 : 11',
 'DMC센트럴자이 : 11',
 '상도역 

### **4) 정보 부족한 행 지우기**
- 찾아도 내용이 안나옴

In [244]:
# # 고양지축 A-2블록 신혼희망타운(공공분양) 행 지우기
# org_apt_data.drop(org_apt_data[org_apt_data['아파트명'].isin(['고양지축 A-2블록 신혼희망타운(공공분양)', '아파트명'])].index, inplace=True)

In [245]:
# drop_check = org_apt_data.loc[org_apt_data['아파트명'] == '고양지축 A-2블록 신혼희망타운(공공분양)']
# drop_check

,아파트명,법정동주소,위도,경도,세대수,임대세대수,최고층,최저층,최대공급면적,최소공급면적,...,노선명_리스트,구,광역시도,소매,음식,교육,장례식장,보건의료,유원지오락,총인구수


In [249]:
# # 전처리 완료 후 csv로 저장
# org_apt_data.to_csv('apartment_rail_v7.csv', index=False)

# # <저장한 csv파일로 구글시트 에서 수행한 작업>
# # - 후에 용적률, 건폐율 부족한 부분 찾아 넣음
# # - 법정동 주소 수정
# # - 아파트명 수정

==============================================================================================================================================

# **1. 데이터 로드**
- 구글시트에서 수정된 데이터로 지하철역과 병원 공원 대학을 수정한 데이터

In [17]:
org_apt_data = pd.read_csv('ver8_apartment_20230822.csv')
print('청약 원본 데이터 shape :', org_apt_data.shape)
org_apt_data.head(1)

청약 원본 데이터 shape : (3133, 41)


,아파트명,법정동주소,위도,경도,세대수,임대세대수,최고층,최저층,최대공급면적,최소공급면적,...,대학,광역,기초,소매,음식,교육,장례식장,보건의료,유원지오락,총인구수
0,올림픽파크 포레온,서울특별시 강동구 둔촌1동 170-1,37.522886,127.140539,12032,1046.0,35.0,20.0,114.42,46.21,...,5,서울,강동구,"12,030","4,934","1,344",19,332,431,"460,141"


# **2. 데이터 확인**

## 1) 형변환이 필요한 컬럼 확인

In [9]:
# #데이터 유형 확인
# org_apt_data.info()

#### **<형변환이 필요한 컬럼>**

**int로 변환할 컬럼명**
- 임대세대수      2899 non-null   float64
- 최고층        3103 non-null   float64
- 최저층        3090 non-null   float64
- 용적률        3133 non-null   float64
- 건폐율        3133 non-null   float64
<!-- - 전용율          3133 non-null   float64 -->
- 방수           3051 non-null   float64
- 욕실수          3051 non-null   float64
<!-- - 지하철역_거리(km)  3133 non-null   float64 -->


**더미화 변환할 컬럼명**
- 현관구조         3133 non-null   object -> '혼합식' '계단식' '복도식'
<!-- - 지하철역         3133 non-null   object  --> 
- 역사명          3133 non-null   object 
- 노선명_리스트      3133 non-null   object -> [''] 대괄호 한에 있는 값 가져오기
- 기초            3133 non-null   object -> '시/군/구'
- 광역         3133 non-null   object -> '서울' '경기' '인천
- 대형건설사        3133 non-null   bool   -> YES, NO

In [14]:
# 데이터 유니크값 확인

# print('1. 현관구조 unique값 :\n', org_apt_data['현관구조'].unique(), '\n')
# print('2. 지하철역 unique값 :\n', org_apt_data['지하철역'].unique(), '\n')
# print('3. 역사명 unique값 :\n', org_apt_data['역사명'].unique(), '\n')
# print('4. 노선명_리스트 unique값 :\n', org_apt_data['노선명_리스트'].unique(), '\n')
# print('5. 구 unique값 :\n', org_apt_data['기초'].unique(), '\n')
# print('6. 광역시도 unique값 :\n', org_apt_data['광역'].unique(), '\n')
# print('7. 광역기초 unique값 :\n', org_apt_data['광역기초'].unique(), '\n')
# print('8. 대형건설사 unique값 :\n', org_apt_data['대형건설사'].unique(), '\n')

## 2) 결측치 확인

In [15]:
# org_apt_data.isna().sum()

#### **<nan값이 있는 컬럼>**
- 임대세대수          234
- 최고층             30
- 최저층             43
- 총아파트동수          24
- 세대평균_주차대수       68
- 방수              82
- 욕실수             82

==============================================================================================================================================

# **3. 데이터 전처리**
- 광역기초 컬럼 만들기
- 결측치 처리
- 더미화
- replace (X)
- 형변환 
- replace (y)

## 1) 광역기초 컬럼 만들기

In [25]:
apt_data = org_apt_data.copy()

In [26]:
# 광역기초 컬럼 만들기
# 광역(서울, 인천, 경기) + '_' + 기초
apt_data['광역기초'] = apt_data['광역'] + '_' + apt_data['기초']
print('(원본)아파트 df의 shape :', apt_data.shape, '\n')
apt_data.head(1)

(원본)아파트 df의 shape : (3133, 42) 



,아파트명,법정동주소,위도,경도,세대수,임대세대수,최고층,최저층,최대공급면적,최소공급면적,...,광역,기초,소매,음식,교육,장례식장,보건의료,유원지오락,총인구수,광역기초
0,올림픽파크 포레온,서울특별시 강동구 둔촌1동 170-1,37.522886,127.140539,12032,1046.0,35.0,20.0,114.42,46.21,...,서울,강동구,"12,030","4,934","1,344",19,332,431,"460,141",서울_강동구


In [27]:
apt_data.columns

Index(['아파트명', '법정동주소', '위도', '경도', '세대수', '임대세대수', '최고층', '최저층', '최대공급면적',
       '최소공급면적', '총아파트동수', '용적률', '건폐율', '세대평균_주차대수', '공급면적', '전용면적', '전용율',
       '방수', '욕실수', '현관구조', '입주예정연도', '공급액(만원)', '대형건설사', '지하철역', '지하철역_거리',
       '역사명', '노선명_리스트', '1차병원', '2차병원', '3차병원', '공원', '대학', '광역', '기초', '소매',
       '음식', '교육', '장례식장', '보건의료', '유원지오락', '총인구수', '광역기초'],
      dtype='object')

## 2) 결측값 처리 
* **null값이 있는 행 삭제**
- 임대세대수          234
- 최고층             30
- 최저층             43
- 총아파트동수          24
- 방수              82
- 욕실수             82

* **평균값으로 fillna**
- 세대평균_주차대수       68 # null값 평균값으로 대체

In [28]:
# null 값이 있는 행 제거
apt_data.dropna(subset=['임대세대수', '최고층', '최저층', '총아파트동수', '방수', '욕실수'], inplace=True)


# 평균값으로 대체
parking_mean = apt_data['세대평균_주차대수'].mean()
# print('세대평균 주차대수 중위값 :', apt_data['세대평균_주차대수'].median())
print('세개평균_주차대수 값의 mean값 : ', parking_mean)
apt_data['세대평균_주차대수'].fillna(parking_mean, inplace=True)


print('원본 행의 수:', org_apt_data.shape[0], '개')
print('na행 drop 후 행의 수 :', apt_data.shape[0], '개')

세개평균_주차대수 값의 mean값 :  1.3466131774057961
원본 행의 수: 3133 개
na행 drop 후 행의 수 : 2828 개


In [29]:
# 결측값 처리 후 확인
apt_data.isna().sum()

아파트명         0
법정동주소        0
위도           0
경도           0
세대수          0
임대세대수        0
최고층          0
최저층          0
최대공급면적       0
최소공급면적       0
총아파트동수       0
용적률          0
건폐율          0
세대평균_주차대수    0
공급면적         0
전용면적         0
전용율          0
방수           0
욕실수          0
현관구조         0
입주예정연도       0
공급액(만원)      0
대형건설사        0
지하철역         0
지하철역_거리      0
역사명          0
노선명_리스트      0
1차병원         0
2차병원         0
3차병원         0
공원           0
대학           0
광역           0
기초           0
소매           0
음식           0
교육           0
장례식장         0
보건의료         0
유원지오락        0
총인구수         0
광역기초         0
dtype: int64

## 3) X 더미화 
**더미화 변환할 컬럼명**
- ~~광역시도         3133 non-null   object -> '서울' '경기' '인천~~
- ~~현관구조         3133 non-null   object -> '혼합식' '계단식' '복도식'~~
- ~~역사명          3133 non-null   object -> get_dummies~~
- ~~구            3133 non-null   object -> '시/군/구'~~
- ~~광역기초~~

### (1) 광역시도 -> 서울 | 경기 | 인천 더미변수로 변환 후 법정동 주소 drop

In [30]:
# 광역시도 -> 서울 | 경기 | 인천 더미변수로 변환 후 법정동 주소 drop
apt_data['서울'] = apt_data['광역'].str.contains('서울').astype(int)
apt_data['경기'] = apt_data['광역'].str.contains('경기').astype(int)
apt_data['인천'] = apt_data['광역'].str.contains('인천').astype(int)

apt = apt_data.drop('광역', axis=1)
apt.head(1) # 데이터 확인

,아파트명,법정동주소,위도,경도,세대수,임대세대수,최고층,최저층,최대공급면적,최소공급면적,...,음식,교육,장례식장,보건의료,유원지오락,총인구수,광역기초,서울,경기,인천
0,올림픽파크 포레온,서울특별시 강동구 둔촌1동 170-1,37.522886,127.140539,12032,1046.0,35.0,20.0,114.42,46.21,...,"4,934","1,344",19,332,431,"460,141",서울_강동구,1,0,0


### (2) 현관구조 -> 혼합식 | 계단식 | 복도식 더미변수로 변환 후 현관구조 drop

In [31]:
# 현관구조 -> 혼합식 | 계단식 | 복도식 더미변수로 변환 후 현관구조 drop
apt['혼합식'] = apt['현관구조'].str.contains('혼합식').astype(int)
apt['계단식'] = apt['현관구조'].str.contains('계단식').astype(int)
apt['복도식'] = apt['현관구조'].str.contains('복도식').astype(int)

apt.drop('현관구조', axis=1, inplace=True)
apt.head(1)

,아파트명,법정동주소,위도,경도,세대수,임대세대수,최고층,최저층,최대공급면적,최소공급면적,...,보건의료,유원지오락,총인구수,광역기초,서울,경기,인천,혼합식,계단식,복도식
0,올림픽파크 포레온,서울특별시 강동구 둔촌1동 170-1,37.522886,127.140539,12032,1046.0,35.0,20.0,114.42,46.21,...,332,431,"460,141",서울_강동구,1,0,0,1,0,0


### (3) 역사명 get_dummies

In [32]:
# 역사명 더미변수 생성
dummies = pd.get_dummies(apt['역사명'])

# apt df와 역사명 더미변수 열 방향으로 합치기 
apt = pd.concat([apt, dummies], axis=1)
apt.head(1)

,아파트명,법정동주소,위도,경도,세대수,임대세대수,최고층,최저층,최대공급면적,최소공급면적,...,파주역,평촌역,평택역,하남풍산역,행신역,화곡역,화서역,화전역,흑석역,흥선역
0,올림픽파크 포레온,서울특별시 강동구 둔촌1동 170-1,37.522886,127.140539,12032,1046.0,35.0,20.0,114.42,46.21,...,0,0,0,0,0,0,0,0,0,0


### (4) 기초(구) get_dummies

In [35]:
# 구 더미변수 생성 
dummies_2 = pd.get_dummies(apt['기초'])

# apt df와 구 더미변수 열 방향으로 합치기 
apt = pd.concat([apt, dummies_2], axis=1)
apt.head(1)

,아파트명,법정동주소,위도,경도,세대수,임대세대수,최고층,최저층,최대공급면적,최소공급면적,...,의정부시,이천시,종로구,중구,중랑구,파주시,평택시,포천시,하남시,화성시
0,올림픽파크 포레온,서울특별시 강동구 둔촌1동 170-1,37.522886,127.140539,12032,1046.0,35.0,20.0,114.42,46.21,...,0,0,0,0,0,0,0,0,0,0


### (5) 광역기초(광역+기초) get_dummies

In [36]:
# 역사명 더미변수 생성
dummies = pd.get_dummies(apt['광역기초'])

# apt df와 역사명 더미변수 열 방향으로 합치기 
apt = pd.concat([apt, dummies], axis=1)
apt.head(1)

,아파트명,법정동주소,위도,경도,세대수,임대세대수,최고층,최저층,최대공급면적,최소공급면적,...,서울_중랑구,인천_강화군,인천_계양구,인천_남동구,인천_동구,인천_미추홀구,인천_부평구,인천_서구,인천_연수구,인천_중구
0,올림픽파크 포레온,서울특별시 강동구 둔촌1동 170-1,37.522886,127.140539,12032,1046.0,35.0,20.0,114.42,46.21,...,0,0,0,0,0,0,0,0,0,0


## 4) replace
- ~~노선명_리스트      3133 non-null   object -> [''] 대괄호 한에 있는 값 가져오기~~
- 수정전 :  "['인천국제공항선', '6호선', '경의중앙선']"
- 수정후 : '인천국제공항선', '6호선', '경의중앙선'

In [37]:
apt['노선명_리스트'].unique()

array(["['9호선']", "['5호선']", "['5호선', '8호선']", "['2호선']", "['8호선']",
       "['8호선', '분당선']", "['분당선']", "['수인선']", "['1호선']", "['신분당선']",
       "['7호선']", "['인천지하철 1호선']", "['우이신설선', '1호선', '2호선']",
       "['1호선', '경의중앙선', '경춘선', '분당선']", "['3호선', '7호선', '9호선']",
       "['3호선']", "['인천지하철 2호선']", "['인천지하철 1호선', '1호선']", "['진접선']",
       "['4호선']", "['6호선']", "['김포골드라인']", "['인천지하철 1호선', '인천국제공항선']",
       "['인천지하철 2호선', '7호선']", "['인천국제공항선']", "['경강선']", "['에버라인']",
       "['인천지하철 2호선', '1호선']", "['인천지하철 1호선', '인천지하철 2호선']", "['서해선']",
       "['의정부선']", "['경의중앙선']", "['우이신설선']", "['수인선', '4호선']", "['경춘선']",
       "['인천국제공항선', '6호선', '경의중앙선']", "['2호선', '7호선']",
       "['경의중앙선', '경춘선']", "['5호선', '경의중앙선']", "['2호선', '5호선']",
       "['5호선', '1호선']", "['신림선']"], dtype=object)

In [38]:
# 정규표현식 사용하여 '', "", []를 공백으로 replace
import re

# 정규표현식 패턴
pattern = r'["\[\]\'\']'

# 정규표현식 패턴을 사용하여 대체
apt['노선명_리스트'] = apt['노선명_리스트'].apply(lambda x: re.sub(pattern, '', x))

In [39]:
apt['노선명_리스트'].unique()

array(['9호선', '5호선', '5호선, 8호선', '2호선', '8호선', '8호선, 분당선', '분당선', '수인선',
       '1호선', '신분당선', '7호선', '인천지하철 1호선', '우이신설선, 1호선, 2호선',
       '1호선, 경의중앙선, 경춘선, 분당선', '3호선, 7호선, 9호선', '3호선', '인천지하철 2호선',
       '인천지하철 1호선, 1호선', '진접선', '4호선', '6호선', '김포골드라인',
       '인천지하철 1호선, 인천국제공항선', '인천지하철 2호선, 7호선', '인천국제공항선', '경강선', '에버라인',
       '인천지하철 2호선, 1호선', '인천지하철 1호선, 인천지하철 2호선', '서해선', '의정부선', '경의중앙선',
       '우이신설선', '수인선, 4호선', '경춘선', '인천국제공항선, 6호선, 경의중앙선', '2호선, 7호선',
       '경의중앙선, 경춘선', '5호선, 경의중앙선', '2호선, 5호선', '5호선, 1호선', '신림선'],
      dtype=object)

## 5) 형변환 
**int로 변환할 컬럼명**
- 대형건설사        3133 non-null   bool -> True (1), False(0)
- 임대세대수        2899 non-null   float64 
- 최고층          3103 non-null   float64 
- 최저층          3090 non-null   float64 
- 총아파트동수       3109 non-null   float64
- 용적률          3133 non-null   float64
- 건폐율          3133 non-null   float64
- 방수           3051 non-null   float64
- 욕실수          3051 non-null   float64

In [43]:
# bool -> int 변환 : True, False
# float -> int 변환 : '임대세대수', '최고층', '최저층', '총아파트동수', '용적률', '건폐율', '방수', '욕실수'

cols = ['대형건설사', '임대세대수', '최고층', '최저층', '총아파트동수', '용적률', '건폐율', '방수', '욕실수']

# float -> int로 변환
apt[cols] = apt[cols].astype(int)

In [44]:
# int 변환 확인
apt[cols].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2828 entries, 0 to 3127
Data columns (total 9 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   대형건설사   2828 non-null   int64
 1   임대세대수   2828 non-null   int64
 2   최고층     2828 non-null   int64
 3   최저층     2828 non-null   int64
 4   총아파트동수  2828 non-null   int64
 5   용적률     2828 non-null   int64
 6   건폐율     2828 non-null   int64
 7   방수      2828 non-null   int64
 8   욕실수     2828 non-null   int64
dtypes: int64(9)
memory usage: 220.9 KB


## 6) 타겟값 replace

In [45]:
apt['공급액(만원)'] = apt['공급액(만원)'].replace(',', '')

In [46]:
# 공급액(만원) int변환 확인
apt['공급액(만원)'].info() 

<class 'pandas.core.series.Series'>
Int64Index: 2828 entries, 0 to 3127
Series name: 공급액(만원)
Non-Null Count  Dtype
--------------  -----
2828 non-null   int64
dtypes: int64(1)
memory usage: 44.2 KB


==============================================================================================================================================

# **4. 회귀분석을 위한 컬럼명 수정**
- 숫자가 맨 앞 -> '1차병원', '2차병원','3차병원','419민주묘지역'
- () _로 바꾸기 -> '지하철역_거리(km)'

In [47]:
# 수정할 컬럼명 확인

all_col = list(apt.columns)
# all_col

In [48]:
# ()가로표기 컬럼명 변경
apt.rename(columns={'지하철역_거리': '지하철역_거리_km'}, inplace=True)
apt.rename(columns={'공급액(만원)': '공급액_만원'}, inplace=True)

# 숫자로 시작하는 컬럼명 변경
apt.rename(columns={'1차병원': '병원수_1차'}, inplace=True)
apt.rename(columns={'2차병원': '병원수_2차'}, inplace=True)
apt.rename(columns={'3차병원': '병원수_3차'}, inplace=True)
apt.rename(columns={'419민주묘지역': '_419민주묘지역'}, inplace=True)

# 컬럼명 변경
apt.rename(columns={'대형건설사': '대형건설사_YES'}, inplace=True)

In [49]:
# 컬럼명 바뀌었는지 확인
apt[['지하철역_거리_km', '공급액_만원', '병원수_1차', '병원수_2차', '병원수_3차', '_419민주묘지역', '대형건설사_YES']].head(1)

,지하철역_거리_km,공급액_만원,병원수_1차,병원수_2차,병원수_3차,_419민주묘지역,대형건설사_YES
0,0.6223,132040,46,16,1,0,1


In [50]:
apt[['지하철역_거리_km', '공급액_만원', '병원수_1차', '병원수_2차', '병원수_3차', '_419민주묘지역', '대형건설사_YES']].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2828 entries, 0 to 3127
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   지하철역_거리_km  2828 non-null   float64
 1   공급액_만원      2828 non-null   int64  
 2   병원수_1차      2828 non-null   int64  
 3   병원수_2차      2828 non-null   int64  
 4   병원수_3차      2828 non-null   int64  
 5   _419민주묘지역   2828 non-null   uint8  
 6   대형건설사_YES   2828 non-null   int64  
dtypes: float64(1), int64(5), uint8(1)
memory usage: 157.4 KB


==============================================================================================================================================

# **5. EDA**
- 탐색적데이터분석은 다른 파일로 하기

==============================================================================================================================================

# **6. 회귀분석**

In [65]:
all_col = list(apt.columns)

# X, y 분리를 위해 all_col에서 공급액_만원 컬럼 지우기
all_col.remove('공급액_만원')

In [66]:
# ols표에 사용할 X 값 ,콤마 제고 and +로 데이터 이어 가져오기

col_item = ''

for item in all_col :
    if item == '공급액_만원' :
        continue
    if all_col[-1] == item :
        col_item += item
    else :
        col_item += item + " + "

print(col_item)

아파트명 + 법정동주소 + 위도 + 경도 + 세대수 + 임대세대수 + 최고층 + 최저층 + 최대공급면적 + 최소공급면적 + 총아파트동수 + 용적률 + 건폐율 + 세대평균_주차대수 + 공급면적 + 전용면적 + 전용율 + 방수 + 욕실수 + 입주예정연도 + 대형건설사_YES + 지하철역 + 지하철역_거리_km + 역사명 + 노선명_리스트 + 병원수_1차 + 병원수_2차 + 병원수_3차 + 공원 + 대학 + 기초 + 소매 + 음식 + 교육 + 장례식장 + 보건의료 + 유원지오락 + 총인구수 + 광역기초 + 서울 + 경기 + 인천 + 혼합식 + 계단식 + 복도식 + _419민주묘지역 + 가양역 + 가좌역 + 가평역 + 갈산역 + 강일역 + 거여역 + 건대입구역 + 걸포북변역 + 경기광주역 + 경인교대입구역 + 경전철의정부역 + 계양역 + 고덕역 + 고속터미널역 + 고진역 + 고촌역 + 광교역 + 광명사거리역 + 광명역 + 구래역 + 구리역 + 구의역 + 국제업무지구역 + 군포역 + 금곡역 + 금릉역 + 금정역 + 길동역 + 까치울역 + 남구로역 + 남부터미널역 + 남한산성입구역 + 녹천역 + 능곡역 + 달미역 + 당고개역 + 덕계역 + 덕소역 + 덕정역 + 도농역 + 도심역 + 도원역 + 독정역 + 돌곶이역 + 동두천중앙역 + 동수역 + 동오역 + 동인천역 + 동천역 + 동춘역 + 둔전역 + 둔촌오륜역 + 등촌역 + 디지털미디어시티역 + 마곡역 + 마석역 + 마전역 + 마천역 + 망우역 + 망포역 + 매교역 + 매탄권선역 + 면목역 + 명지대역 + 명학역 + 모란역 + 문산역 + 문학경기장역 + 미아역 + 박촌역 + 발산역 + 백운역 + 범계역 + 범골역 + 별내역 + 병점역 + 보평역 + 봉천역 + 부개역 + 부발역 + 부천역 + 부평역 + 산성역 + 삼가역 + 삼동역 + 삼성역 + 삼양사거리역 + 상갈역 + 상도역 + 상동역 + 상일동역 + 상현역 + 서동탄역 + 서울대벤처타운역 + 서울대입구역 + 서정리역 + 석남역 + 석바위시장역 + 선부역 + 성환역 + 세류

## 1-1) 1차 회귀분석 
- 모든 피처(X) 사용

In [67]:
# 1차 ols표 -> X 모두 사용

import statsmodels.formula. api as smf

# 포함하지 않을 X = ['아파트명','법정동주소','지하철역','역사명','노선명_리스트','기초','광역기초']
formula = '공급액_만원 ~ 위도 + 경도 + 세대수 + 임대세대수 + 최고층 + 최저층 + 최대공급면적 + 최소공급면적 + 총아파트동수 + 용적률 + 건폐율 + 세대평균_주차대수 + 공급면적 + 전용면적 + 전용율 + 방수 + 욕실수 + 입주예정연도 + 대형건설사_YES + 지하철역_거리_km + 병원수_1차 + 병원수_2차 + 병원수_3차 + 공원 + 대학 + 소매 + 음식 + 교육 + 장례식장 + 보건의료 + 유원지오락 + 총인구수  + 서울 + 경기 + 인천 + 혼합식 + 계단식 + 복도식 + _419민주묘지역 + 가양역 + 가좌역 + 가평역 + 갈산역 + 강일역 + 거여역 + 건대입구역 + 걸포북변역 + 경기광주역 + 경인교대입구역 + 경전철의정부역 + 계양역 + 고덕역 + 고속터미널역 + 고진역 + 고촌역 + 광교역 + 광명사거리역 + 광명역 + 구래역 + 구리역 + 구의역 + 국제업무지구역 + 군포역 + 금곡역 + 금릉역 + 금정역 + 길동역 + 까치울역 + 남구로역 + 남부터미널역 + 남한산성입구역 + 녹천역 + 능곡역 + 달미역 + 당고개역 + 덕계역 + 덕소역 + 덕정역 + 도농역 + 도심역 + 도원역 + 독정역 + 돌곶이역 + 동두천중앙역 + 동수역 + 동오역 + 동인천역 + 동천역 + 동춘역 + 둔전역 + 둔촌오륜역 + 등촌역 + 디지털미디어시티역 + 마곡역 + 마석역 + 마전역 + 마천역 + 망우역 + 망포역 + 매교역 + 매탄권선역 + 면목역 + 명지대역 + 명학역 + 모란역 + 문산역 + 문학경기장역 + 미아역 + 박촌역 + 발산역 + 백운역 + 범계역 + 범골역 + 별내역 + 병점역 + 보평역 + 봉천역 + 부개역 + 부발역 + 부천역 + 부평역 + 산성역 + 삼가역 + 삼동역 + 삼성역 + 삼양사거리역 + 상갈역 + 상도역 + 상동역 + 상일동역 + 상현역 + 서동탄역 + 서울대벤처타운역 + 서울대입구역 + 서정리역 + 석남역 + 석바위시장역 + 선부역 + 성환역 + 세류역 + 세마역 + 센트럴파크역 + 소새울역 + 소요산역 + 송도달빛축제공원역 + 송정역 + 송탄역 + 수색역 + 숭의역 + 시민공원역 + 시흥시청역 + 신길역 + 신길온천역 + 신답역 + 신반포역 + 신설동역 + 신원역 + 신정네거리역 + 신천역 + 쌍문역 + 아시아드경기장역 + 안산역 + 안암역 + 안양역 + 야당역 + 야목역 + 양주역 + 양천향교역 + 양촌역 + 양평역 + 어정역 + 어천역 + 여주역 + 역곡역 + 연수역 + 영등포구청역 + 영종역 + 예술회관역 + 오목천역 + 오빈역 + 오산대역 + 오산역 + 완정역 + 왕길역 + 외대앞역 + 용마산역 + 용문역 + 운동장송담대역 + 운서역 + 운정역 + 원당역 + 을지로4가역 + 응암역 + 의정부시청역 + 의정부역 + 의정부중앙역 + 이천역 + 인덕원역 + 인천대입구역 + 인천시청역 + 인하대역 + 일원역 + 작전역 + 장한평역 + 정부과천청사역 + 정왕역 + 제물포역 + 주안역 + 죽전역 + 중동역 + 중앙역 + 중화역 + 지식정보단지역 + 지제역 + 지행역 + 진위역 + 진접역 + 창신역 + 천왕역 + 천호역 + 철산역 + 청라국제도시역 + 청량리역 + 청명역 + 초월역 + 탑석역 + 테크노파크역 + 퇴계원역 + 파주역 + 평촌역 + 평택역 + 하남풍산역 + 행신역 + 화곡역 + 화서역 + 화전역 + 흑석역 + 흥선역 + 가평군 + 강남구 + 강동구 + 강북구 + 강서구 + 강화군 + 계양구 + 고양시 + 과천시 + 관악구 + 광명시 + 광주시 + 광진구 + 구로구 + 구리시 + 군포시 + 김포시 + 남동구 + 남양주시 + 노원구 + 도봉구 + 동구 + 동대문구 + 동두천시 + 동작구 + 미추홀구 + 부천시 + 부평구 + 서구 + 서대문구 + 서초구 + 성남시 + 성동구 + 성북구 + 송파구 + 수원시 + 시흥시 + 안산시 + 안성시 + 안양시 + 양주시 + 양천구 + 양평군 + 여주시 + 연수구 + 연천군 + 영등포구 + 오산시 + 용인시 + 은평구 + 의왕시 + 의정부시 + 이천시 + 종로구 + 중구 + 중랑구 + 파주시 + 평택시 + 포천시 + 하남시 + 화성시 + 경기_가평군 + 경기_고양시 + 경기_과천시 + 경기_광명시 + 경기_광주시 + 경기_구리시 + 경기_군포시 + 경기_김포시 + 경기_남양주시 + 경기_동두천시 + 경기_부천시 + 경기_성남시 + 경기_수원시 + 경기_시흥시 + 경기_안산시 + 경기_안성시 + 경기_안양시 + 경기_양주시 + 경기_양평군 + 경기_여주시 + 경기_연천군 + 경기_오산시 + 경기_용인시 + 경기_의왕시 + 경기_의정부시 + 경기_이천시 + 경기_파주시 + 경기_평택시 + 경기_포천시 + 경기_하남시 + 경기_화성시 + 서울_강남구 + 서울_강동구 + 서울_강북구 + 서울_강서구 + 서울_관악구 + 서울_광진구 + 서울_구로구 + 서울_노원구 + 서울_도봉구 + 서울_동대문구 + 서울_동작구 + 서울_서대문구 + 서울_서초구 + 서울_성동구 + 서울_성북구 + 서울_송파구 + 서울_양천구 + 서울_영등포구 + 서울_은평구 + 서울_종로구 + 서울_중구 + 서울_중랑구 + 인천_강화군 + 인천_계양구 + 인천_남동구 + 인천_동구 + 인천_미추홀구 + 인천_부평구 + 인천_서구 + 인천_연수구 + 인천_중구'
results = smf.ols(formula, apt).fit()

print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                 공급액_만원   R-squared:                       0.924
Model:                            OLS   Adj. R-squared:                  0.917
Method:                 Least Squares   F-statistic:                     123.6
Date:                Sat, 30 Sep 2023   Prob (F-statistic):               0.00
Time:                        23:13:25   Log-Likelihood:                -29571.
No. Observations:                2828   AIC:                         5.965e+04
Df Residuals:                    2573   BIC:                         6.117e+04
Df Model:                         254                                         
Covariance Type:            nonrobust                                         
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
Intercept         -1.529e+05   8.01e+0

In [68]:
# 회귀 모델에 사용할 독립 변수 선택
independent_vars = ['위도','경도','세대수','임대세대수','최고층','최저층','최대공급면적','최소공급면적','총아파트동수', '대형건설사_YES',
                    '용적률','건폐율','세대평균_주차대수','공급면적','전용면적','전용율','방수','욕실수',
                    '지하철역_거리_km','병원수_1차','병원수_2차','병원수_3차','공원','대학',
                    '소매','음식','교육','장례식장','보건의료','유원지오락', '총인구수','서울','경기','인천','혼합식','계단식','복도식',
                    '_419민주묘지역','가양역','가좌역','가평역','갈산역','강일역','거여역','건대입구역','걸포북변역','경기광주역','경인교대입구역','경전철의정부역','계양역',
                    '고덕역','고속터미널역','고진역','고촌역','광교역','광명사거리역','광명역','구래역','구리역','구의역','국제업무지구역','군포역','금릉역','금정역','길동역','까치울역',
                    '남구로역','남부터미널역','남한산성입구역','능곡역','달미역','당고개역','덕계역','덕소역','덕정역','도농역','도심역','도원역','독정역','돌곶이역',
                    '동두천중앙역','동수역','동오역','동인천역','동천역','동춘역','둔전역','둔촌오륜역','등촌역','디지털미디어시티역','마곡역','마전역','마천역','망우역','망포역',
                    '매교역','매탄권선역','면목역','명지대역','명학역','모란역','문산역','문학경기장역','미아역','박촌역','발산역','백운역','범계역','범골역','별내역','병점역',
                    '보평역','봉천역','부개역','부천역','부평역','산성역','삼가역','삼동역','삼성역','상갈역','상도역','상일동역','상현역','서동탄역','서울대벤처타운역','서정리역',
                    '석남역','석바위시장역','성환역','세마역','센트럴파크역','소새울역','소요산역','송도달빛축제공원역','송정역','송탄역','수색역','숭의역','시민공원역','시흥시청역',
                    '신길온천역','신답역','신반포역','신설동역','신원역','신정네거리역','신천역','쌍문역','아시아드경기장역','안산역','안양역','야당역','야목역','양주역','양천향교역','양촌역','양평역',
                    '어정역','어천역','여주역','역곡역','연수역','영등포구청역','영종역','오목천역','오빈역','오산대역','오산역','완정역','왕길역','외대앞역','용마산역','용문역',
                    '운동장송담대역','운서역','운정역','을지로4가역','응암역','의정부시청역','의정부역','의정부중앙역','이천역','인덕원역','인천대입구역','인하대역','일원역',
                    '작전역','장한평역','정왕역','제물포역','주안역','죽전역','중동역','중앙역','중화역','지식정보단지역','지제역','지행역','진위역','진접역','창신역','천호역','철산역',
                    '청라국제도시역','청량리역','청명역','초월역','테크노파크역','퇴계원역','파주역','평촌역','평택역','하남풍산역','행신역','화곡역','화서역','화전역','흑석역','흥선역',
                    '가평군','강남구','강동구','강북구','강서구','강화군','계양구','고양시','과천시','관악구','광명시','광주시','광진구','구로구','구리시','군포시','김포시',
                    '남양주시','노원구','도봉구','동구','동대문구','동두천시','동작구','미추홀구','부천시','부평구','서구','서대문구','중구','서초구','성남시','성동구','성북구',
                    '송파구','수원시','시흥시','안산시','안성시','안양시','양주시','양천구','양평군','여주시','연수구','연천군','영등포구','오산시','용인시','은평구',
                    '의왕시','의정부시','이천시','중구','종로구','중랑구','파주시','평택시','포천시','하남시','화성시',
                    '경기_고양시','경기_과천시','경기_광명시','경기_광주시','경기_구리시','경기_군포시','경기_김포시','경기_남양주시','경기_동두천시','경기_부천시','경기_성남시','경기_수원시',
                    '경기_시흥시','경기_안산시','경기_안성시','경기_안양시','경기_양주시','경기_양평군','경기_여주시','경기_연천군','경기_오산시','경기_용인시','경기_의왕시','경기_의정부시',
                    '경기_이천시','경기_파주시','경기_평택시','경기_포천시','경기_하남시','경기_화성시','서울_강남구','서울_강동구','서울_강북구','서울_강서구','서울_관악구','서울_광진구','서울_구로구',
                    '서울_노원구','서울_도봉구','서울_동대문구','서울_동작구','서울_서대문구','서울_서초구','서울_성동구','서울_성북구','서울_송파구','서울_양천구','서울_영등포구','서울_은평구',
                    '서울_종로구','서울_중구','서울_중랑구','인천_강화군','인천_계양구','인천_남동구','인천_동구','인천_미추홀구','인천_부평구','인천_서구','인천_연수구','인천_중구']

**<다중공선성>**
- VIF가 높을수록 해당 변수는 다른 변수들과 강한 상관관계가 있을 가능성이 높음
- VIF value : 
   - VIF >= 10 : 다중공선성의 가능성 존재
   - VIF >= 5 : 주의 필요

## **1-2) 1차 회귀분석 다중공선성(오류)** 
- 모든 피처(X) 사용

In [1166]:
# <object 형이 있어서 오류남>

# import statsmodels.api as sm
# from statsmodels.stats.outliers_influence import variance_inflation_factor

# # 회귀 모델 적합
# X = sm.add_constant(apt[independent_vars])
# y = apt['공급액_만원']
# model = sm.OLS(y, X)
# results = model.fit()

# # VIF 계산
# vif = pd.DataFrame()
# vif["Variable"] = X.columns
# vif["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]

# print(vif)

ValueError: Pandas data cast to numpy dtype of object. Check input data with np.asarray(data).

#### **(오류) ValueError: Pandas data cast to numpy dtype of object. Check input data with np.asarray(data)**

## **오류 해결을 위한 전처리**
- object -> int

In [69]:
# object형이 있어서 오류 나는 것 같아 확인
apt.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2828 entries, 0 to 3127
Columns: 369 entries, 아파트명 to 인천_중구
dtypes: float64(9), int64(24), object(13), uint8(323)
memory usage: 1.9+ MB


In [70]:
# col이 object형인 것만 뽑아보기

object_columns = apt.select_dtypes(include=['object']).columns
object_data = apt[object_columns]
object_data.head(1)

# 아파트명, 법정동주소, 지하철역, 역사명, 노선명_리스트, 기초, 광역기초 는 X에 포함하지 않음.
# 소매, 음식, 교육, 보건의료, 유원지오락, 총인구수 -> int로 바꾸기

,아파트명,법정동주소,지하철역,역사명,노선명_리스트,기초,소매,음식,교육,보건의료,유원지오락,총인구수,광역기초
0,올림픽파크 포레온,서울특별시 강동구 둔촌1동 170-1,둔촌오륜역,둔촌오륜역,9호선,강동구,"12,030","4,934","1,344",332,431,"460,141",서울_강동구


#### **소매, 음식, 교육, 보건의료, 유원지오락, 총인구수 -> int로 바꾸기**

In [71]:
apt[['소매']].info()
apt['소매'] = apt['소매'].str.replace(",", "").astype(int)
apt[['소매']].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2828 entries, 0 to 3127
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   소매      2828 non-null   object
dtypes: object(1)
memory usage: 44.2+ KB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 2828 entries, 0 to 3127
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   소매      2828 non-null   int64
dtypes: int64(1)
memory usage: 44.2 KB


In [72]:
apt[['음식']].info()
apt['음식'] = apt['음식'].str.replace(",", "").astype(int)
apt[['음식']].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2828 entries, 0 to 3127
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   음식      2828 non-null   object
dtypes: object(1)
memory usage: 44.2+ KB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 2828 entries, 0 to 3127
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   음식      2828 non-null   int64
dtypes: int64(1)
memory usage: 44.2 KB


In [76]:
apt[['교육']].info()
apt['교육'] = apt['교육'].str.replace(",", "").astype(int)
apt[['교육']].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2828 entries, 0 to 3127
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   교육      2828 non-null   object
dtypes: object(1)
memory usage: 44.2+ KB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 2828 entries, 0 to 3127
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   교육      2828 non-null   int64
dtypes: int64(1)
memory usage: 44.2 KB


In [73]:
apt[['보건의료']].info()
apt['보건의료'] = apt['보건의료'].str.replace(",", "").astype(int)
apt[['보건의료']].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2828 entries, 0 to 3127
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   보건의료    2828 non-null   object
dtypes: object(1)
memory usage: 44.2+ KB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 2828 entries, 0 to 3127
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   보건의료    2828 non-null   int64
dtypes: int64(1)
memory usage: 44.2 KB


In [74]:
apt[['유원지오락']].info()
apt['유원지오락'] = apt['유원지오락'].str.replace(",", "").astype(int)
apt[['유원지오락']].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2828 entries, 0 to 3127
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   유원지오락   2828 non-null   object
dtypes: object(1)
memory usage: 44.2+ KB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 2828 entries, 0 to 3127
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   유원지오락   2828 non-null   int64
dtypes: int64(1)
memory usage: 44.2 KB


In [75]:
apt[['총인구수']].info()
apt['총인구수'] = apt['총인구수'].str.replace(",", "").astype(int)
apt[['총인구수']].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2828 entries, 0 to 3127
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   총인구수    2828 non-null   object
dtypes: object(1)
memory usage: 44.2+ KB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 2828 entries, 0 to 3127
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   총인구수    2828 non-null   int64
dtypes: int64(1)
memory usage: 44.2 KB


In [77]:
# 바뀐지 확인
apt[['소매', '음식', '교육', '보건의료', '유원지오락', '총인구수']].head(1)

,소매,음식,교육,보건의료,유원지오락,총인구수
0,12030,4934,1344,332,431,460141


## **1-3) 1차 회귀분석 다중공선성 (오류수정)** 
- 모든 피처(X) 사용

In [78]:
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor

# 회귀 모델 적합
X = sm.add_constant(apt[independent_vars])
y = apt['공급액_만원']
model = sm.OLS(y, X)
results = model.fit()

# VIF 계산
vif = pd.DataFrame()
vif["Variable"] = X.columns
vif["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]

print(vif)

/Users/jang-yeji/miniconda3/envs/dataanalysis/lib/python3.8/site-packages/statsmodels/regression/linear_model.py:1781: RuntimeWarning: divide by zero encountered in scalar divide
  return 1 - self.ssr/self.centered_tss
/Users/jang-yeji/miniconda3/envs/dataanalysis/lib/python3.8/site-packages/statsmodels/stats/outliers_influence.py:198: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)


    Variable         VIF
0      const    0.000000
1         위도  196.472851
2         경도  112.111215
3        세대수   46.590215
4      임대세대수   31.845740
..       ...         ...
338  인천_미추홀구         inf
339   인천_부평구         inf
340    인천_서구         inf
341   인천_연수구         inf
342    인천_중구         inf

[343 rows x 2 columns]


**<1차 VIF 결과>**
- 10 이상 값

In [79]:
# GPT 코드 <VIF가 10 이상인 것만 뽑아내기>

high_vif_vars = []
high_vif_values = []
threshold = 10  #임계값 설정

for i in range(1, X.shape[1]):  # 첫 번째 컬럼은 상수항이므로 제외
    vif = variance_inflation_factor(X.values, i)
    if vif >= threshold:
        high_vif_vars.append(X.columns[i])
        high_vif_values.append(vif)

print("Variables with VIF >= {}:".format(threshold))
for var, vif in zip(high_vif_vars, high_vif_values):
    print("{}: {:.2f}".format(var, vif))

/Users/jang-yeji/miniconda3/envs/dataanalysis/lib/python3.8/site-packages/statsmodels/stats/outliers_influence.py:198: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)


Variables with VIF >= 10:
위도: 196.47
경도: 112.11
세대수: 46.59
임대세대수: 31.85
총아파트동수: 31.68
공급면적: 860.21
전용면적: 925.33
전용율: 18.85
대학: 13.99
소매: inf
음식: inf
교육: inf
장례식장: inf
보건의료: inf
유원지오락: inf
총인구수: inf
서울: inf
경기: inf
인천: inf
혼합식: inf
계단식: inf
복도식: inf
가좌역: inf
가평역: 19.46
강일역: inf
거여역: inf
건대입구역: inf
걸포북변역: inf
경기광주역: 11.73
고덕역: inf
고속터미널역: inf
광명사거리역: inf
광명역: inf
구래역: inf
구리역: inf
구의역: inf
국제업무지구역: 34.84
군포역: inf
금릉역: inf
금정역: inf
길동역: inf
남부터미널역: inf
남한산성입구역: inf
당고개역: inf
덕계역: 23.49
덕정역: 15.80
도원역: 24.29
독정역: 14.74
동춘역: 27.66
둔촌오륜역: inf
등촌역: inf
디지털미디어시티역: inf
마곡역: inf
마전역: 12.90
마천역: inf
망우역: inf
면목역: inf
명학역: inf
모란역: inf
문산역: inf
발산역: inf
범계역: inf
산성역: inf
삼성역: inf
상도역: inf
상일동역: inf
서정리역: 18.14
센트럴파크역: 11.44
소새울역: 10.74
소요산역: inf
송도달빛축제공원역: 23.62
송정역: inf
수색역: inf
시흥시청역: inf
신길온천역: inf
신답역: inf
신반포역: inf
신설동역: inf
신원역: inf
신정네거리역: inf
신천역: inf
안양역: inf
야당역: inf
양촌역: inf
양평역: inf
어천역: 10.17
여주역: 15.62
영종역: 42.16
오빈역: inf
오산대역: 12.44
오산역: 14.26
외대앞역: inf
용마산역: inf
용문역: inf
운서역: 44.31

## **2-1) 2차 회귀분석** 

In [80]:
# 2차 ols표 

import statsmodels.formula. api as smf

# 포함하지 않을 X = ['아파트명','법정동주소','지하철역','역사명','노선명_리스트','기초','광역기초' 
#                  위도 + 경도 + 최대공급면적 + 최소공급면적 + 전용면적 + 전용율 + 입주예정연도
#                  경기 + 인천 + 혼합식 + 계단식 + 복도식 + 가평군 + 강남구 + 강동구 + 강북구 + 강서구 + 강화군 + 계양구 + 고양시 + 과천시 + 관악구 + 광명시 + 광주시
#                  광진구 + 구로구 + 구리시 + 군포시 + 김포시 + 남동구 + 남양주시 + 노원구 + 도봉구 + 동구 + 동대문구 + 동두천시 + 동작구 + 
#                  미추홀구 + 부천시 + 부평구 + 서구 + 서대문구 + 서초구 + 성남시 + 성동구 + 성북구 + 송파구 + 수원시 + 시흥시 + 
#                  안산시 + 안성시 + 안양시 + 양주시 + 양천구 + 양평군 + 여주시 + 연수구 + 연천군 + 영등포구 + 오산시 + 용인시 + 은평구 + 의왕시 + 의정부시 + 이천시 + 
#                  종로구 + 중구 + 중랑구 + 파주시 + 평택시 + 포천시 + 하남시 + 화성시]

formula = '공급액_만원 ~ 세대수 + 임대세대수 + 최고층 + 최저층 + 총아파트동수 + 용적률 + 건폐율 + 세대평균_주차대수 + 공급면적 + 방수 + 욕실수 + 대형건설사_YES + 지하철역_거리_km + 병원수_1차 + 병원수_2차 + 병원수_3차 + 공원 + 대학 + 소매 + 음식 + 교육 + 장례식장 + 보건의료 + 유원지오락 + 총인구수  + 서울 + _419민주묘지역 + 가양역 + 가좌역 + 가평역 + 갈산역 + 강일역 + 거여역 + 건대입구역 + 걸포북변역 + 경기광주역 + 경인교대입구역 + 경전철의정부역 + 계양역 + 고덕역 + 고속터미널역 + 고진역 + 고촌역 + 광교역 + 광명사거리역 + 광명역 + 구래역 + 구리역 + 구의역 + 국제업무지구역 + 군포역 + 금곡역 + 금릉역 + 금정역 + 길동역 + 까치울역 + 남구로역 + 남부터미널역 + 남한산성입구역 + 녹천역 + 능곡역 + 달미역 + 당고개역 + 덕계역 + 덕소역 + 덕정역 + 도농역 + 도심역 + 도원역 + 독정역 + 돌곶이역 + 동두천중앙역 + 동수역 + 동오역 + 동인천역 + 동천역 + 동춘역 + 둔전역 + 둔촌오륜역 + 등촌역 + 디지털미디어시티역 + 마곡역 + 마석역 + 마전역 + 마천역 + 망우역 + 망포역 + 매교역 + 매탄권선역 + 면목역 + 명지대역 + 명학역 + 모란역 + 문산역 + 문학경기장역 + 미아역 + 박촌역 + 발산역 + 백운역 + 범계역 + 범골역 + 별내역 + 병점역 + 보평역 + 봉천역 + 부개역 + 부발역 + 부천역 + 부평역 + 산성역 + 삼가역 + 삼동역 + 삼성역 + 삼양사거리역 + 상갈역 + 상도역 + 상동역 + 상일동역 + 상현역 + 서동탄역 + 서울대벤처타운역 + 서울대입구역 + 서정리역 + 석남역 + 석바위시장역 + 선부역 + 성환역 + 세류역 + 세마역 + 센트럴파크역 + 소새울역 + 소요산역 + 송도달빛축제공원역 + 송정역 + 송탄역 + 수색역 + 숭의역 + 시민공원역 + 시흥시청역 + 신길역 + 신길온천역 + 신답역 + 신반포역 + 신설동역 + 신원역 + 신정네거리역 + 신천역 + 쌍문역 + 아시아드경기장역 + 안산역 + 안암역 + 안양역 + 야당역 + 야목역 + 양주역 + 양천향교역 + 양촌역 + 양평역 + 어정역 + 어천역 + 여주역 + 역곡역 + 연수역 + 영등포구청역 + 영종역 + 예술회관역 + 오목천역 + 오빈역 + 오산대역 + 오산역 + 완정역 + 왕길역 + 외대앞역 + 용마산역 + 용문역 + 운동장송담대역 + 운서역 + 운정역 + 원당역 + 을지로4가역 + 응암역 + 의정부시청역 + 의정부역 + 의정부중앙역 + 이천역 + 인덕원역 + 인천대입구역 + 인천시청역 + 인하대역 + 일원역 + 작전역 + 장한평역 + 정부과천청사역 + 정왕역 + 제물포역 + 주안역 + 죽전역 + 중동역 + 중앙역 + 중화역 + 지식정보단지역 + 지제역 + 지행역 + 진위역 + 진접역 + 창신역 + 천왕역 + 천호역 + 철산역 + 청라국제도시역 + 청량리역 + 청명역 + 초월역 + 탑석역 + 테크노파크역 + 퇴계원역 + 파주역 + 평촌역 + 평택역 + 하남풍산역 + 행신역 + 화곡역 + 화서역 + 화전역 + 흑석역 + 흥선역 + 경기_가평군 + 경기_고양시 + 경기_과천시 + 경기_광명시 + 경기_광주시 + 경기_구리시 + 경기_군포시 + 경기_김포시 + 경기_남양주시 + 경기_동두천시 + 경기_부천시 + 경기_성남시 + 경기_수원시 + 경기_시흥시 + 경기_안산시 + 경기_안성시 + 경기_안양시 + 경기_양주시 + 경기_양평군 + 경기_여주시 + 경기_연천군 + 경기_오산시 + 경기_용인시 + 경기_의왕시 + 경기_의정부시 + 경기_이천시 + 경기_파주시 + 경기_평택시 + 경기_포천시 + 경기_하남시 + 경기_화성시 + 서울_강남구 + 서울_강동구 + 서울_강북구 + 서울_강서구 + 서울_관악구 + 서울_광진구 + 서울_구로구 + 서울_노원구 + 서울_도봉구 + 서울_동대문구 + 서울_동작구 + 서울_서대문구 + 서울_서초구 + 서울_성동구 + 서울_성북구 + 서울_송파구 + 서울_양천구 + 서울_영등포구 + 서울_은평구 + 서울_종로구 + 서울_중구 + 서울_중랑구 + 인천_강화군 + 인천_계양구 + 인천_남동구 + 인천_동구 + 인천_미추홀구 + 인천_부평구 + 인천_서구 + 인천_연수구 + 인천_중구'
results = smf.ols(formula, apt).fit()

print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                 공급액_만원   R-squared:                       0.922
Model:                            OLS   Adj. R-squared:                  0.914
Method:                 Least Squares   F-statistic:                     124.4
Date:                Sat, 30 Sep 2023   Prob (F-statistic):               0.00
Time:                        23:16:28   Log-Likelihood:                -29615.
No. Observations:                2828   AIC:                         5.972e+04
Df Residuals:                    2582   BIC:                         6.118e+04
Df Model:                         245                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept   -3.06e+04   2472.436    -12.377      0.0

## **2-2) 2차 회귀분석 VIF** 

In [81]:
# 회귀 모델에 사용할 독립 변수 선택
independent_vars = ['세대수','임대세대수','최고층','최저층','총아파트동수', '대형건설사_YES',
                    '용적률','건폐율','세대평균_주차대수','공급면적','방수','욕실수',
                    '지하철역_거리_km','병원수_1차','병원수_2차','병원수_3차','공원','대학',
                    '소매','음식','교육','장례식장','보건의료','유원지오락', '총인구수','서울',
                    '_419민주묘지역','가양역','가좌역','가평역','갈산역','강일역','거여역','건대입구역','걸포북변역','경기광주역','경인교대입구역','경전철의정부역','계양역',
                    '고덕역','고속터미널역','고진역','고촌역','광교역','광명사거리역','광명역','구래역','구리역','구의역','국제업무지구역','군포역','금릉역','금정역','길동역','까치울역',
                    '남구로역','남부터미널역','남한산성입구역','능곡역','달미역','당고개역','덕계역','덕소역','덕정역','도농역','도심역','도원역','독정역','돌곶이역',
                    '동두천중앙역','동수역','동오역','동인천역','동천역','동춘역','둔전역','둔촌오륜역','등촌역','디지털미디어시티역','마곡역','마전역','마천역','망우역','망포역',
                    '매교역','매탄권선역','면목역','명지대역','명학역','모란역','문산역','문학경기장역','미아역','박촌역','발산역','백운역','범계역','범골역','별내역','병점역',
                    '보평역','봉천역','부개역','부천역','부평역','산성역','삼가역','삼동역','삼성역','상갈역','상도역','상일동역','상현역','서동탄역','서울대벤처타운역','서정리역',
                    '석남역','석바위시장역','성환역','세마역','센트럴파크역','소새울역','소요산역','송도달빛축제공원역','송정역','송탄역','수색역','숭의역','시민공원역','시흥시청역',
                    '신길온천역','신답역','신반포역','신설동역','신원역','신정네거리역','신천역','쌍문역','아시아드경기장역','안산역','안양역','야당역','야목역','양주역','양천향교역','양촌역','양평역',
                    '어정역','어천역','여주역','역곡역','연수역','영등포구청역','영종역','오목천역','오빈역','오산대역','오산역','완정역','왕길역','외대앞역','용마산역','용문역',
                    '운동장송담대역','운서역','운정역','을지로4가역','응암역','의정부시청역','의정부역','의정부중앙역','이천역','인덕원역','인천대입구역','인하대역','일원역',
                    '작전역','장한평역','정왕역','제물포역','주안역','죽전역','중동역','중앙역','중화역','지식정보단지역','지제역','지행역','진위역','진접역','창신역','천호역','철산역',
                    '청라국제도시역','청량리역','청명역','초월역','테크노파크역','퇴계원역','파주역','평촌역','평택역','하남풍산역','행신역','화곡역','화서역','화전역','흑석역','흥선역',
                    '경기_고양시','경기_과천시','경기_광명시','경기_광주시','경기_구리시','경기_군포시','경기_김포시','경기_남양주시','경기_동두천시','경기_부천시','경기_성남시','경기_수원시',
                    '경기_시흥시','경기_안산시','경기_안성시','경기_안양시','경기_양주시','경기_양평군','경기_여주시','경기_연천군','경기_오산시','경기_용인시','경기_의왕시','경기_의정부시',
                    '경기_이천시','경기_파주시','경기_평택시','경기_포천시','경기_하남시','경기_화성시','서울_강남구','서울_강동구','서울_강북구','서울_강서구','서울_관악구','서울_광진구','서울_구로구',
                    '서울_노원구','서울_도봉구','서울_동대문구','서울_동작구','서울_서대문구','서울_서초구','서울_성동구','서울_성북구','서울_송파구','서울_양천구','서울_영등포구','서울_은평구',
                    '서울_종로구','서울_중구','서울_중랑구','인천_강화군','인천_계양구','인천_남동구','인천_동구','인천_미추홀구','인천_부평구','인천_서구','인천_연수구','인천_중구']

# 회귀 모델 적합
X = sm.add_constant(apt[independent_vars])
y = apt['공급액_만원']
model = sm.OLS(y, X)
results = model.fit()

# # VIF 계산
# vif = pd.DataFrame()
# vif["Variable"] = X.columns
# vif["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]

# print(vif)

In [82]:
# GPT 코드 <VIF가 10 이상인 것만 뽑아내기>

high_vif_vars = []
high_vif_values = []
threshold = 10  #임계값 설정

for i in range(1, X.shape[1]):  # 첫 번째 컬럼은 상수항이므로 제외
    vif = variance_inflation_factor(X.values, i)
    if vif >= threshold:
        high_vif_vars.append(X.columns[i])
        high_vif_values.append(vif)

print("Variables with VIF >= {}:".format(threshold))
for var, vif in zip(high_vif_vars, high_vif_values):
    print("{}: {:.2f}".format(var, vif))

/Users/jang-yeji/miniconda3/envs/dataanalysis/lib/python3.8/site-packages/statsmodels/stats/outliers_influence.py:198: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)


Variables with VIF >= 10:
세대수: 44.06
임대세대수: 28.05
총아파트동수: 30.67
대학: 13.71
소매: inf
음식: inf
교육: inf
장례식장: inf
보건의료: inf
유원지오락: inf
총인구수: inf
서울: inf
가좌역: inf
가평역: 10.11
강일역: inf
거여역: inf
건대입구역: inf
걸포북변역: inf
경기광주역: 10.34
고덕역: inf
고속터미널역: inf
광명사거리역: inf
광명역: inf
구래역: inf
구리역: inf
구의역: inf
국제업무지구역: 31.55
군포역: inf
금릉역: inf
금정역: inf
길동역: inf
남부터미널역: inf
남한산성입구역: inf
당고개역: inf
덕계역: 19.95
덕정역: 13.07
동춘역: 25.44
둔촌오륜역: inf
등촌역: inf
디지털미디어시티역: inf
마곡역: inf
마천역: inf
망우역: inf
면목역: inf
명학역: inf
모란역: inf
문산역: inf
발산역: inf
범계역: inf
산성역: inf
삼성역: inf
상도역: inf
상일동역: inf
센트럴파크역: 10.59
소요산역: inf
송도달빛축제공원역: 21.55
송정역: inf
수색역: inf
시흥시청역: inf
신길온천역: inf
신답역: inf
신반포역: inf
신설동역: inf
신원역: inf
신정네거리역: inf
신천역: inf
안양역: inf
야당역: inf
양촌역: inf
양평역: inf
여주역: 10.08
영종역: 36.35
오빈역: inf
외대앞역: inf
용마산역: inf
용문역: inf
운서역: 36.61
운정역: inf
을지로4가역: inf
응암역: inf
인천대입구역: 64.09
일원역: inf
장한평역: inf
정왕역: inf
중화역: inf
창신역: inf
천호역: inf
철산역: inf
청량리역: inf
초월역: 17.56
파주역: inf
평촌역: inf
평택역: 11.52
하남풍산역: inf
화곡역: inf
흑석역: inf
경기_고양

## **3-1) 3차 회귀분석** 

In [83]:
# 3차 ols표 

import statsmodels.formula. api as smf

# 포함하지 않을 X = ['아파트명','법정동주소','지하철역','역사명','노선명_리스트','기초','광역기초' 
#                  위도 + 경도 + 최대공급면적 + 최소공급면적 + 전용면적 + 전용율 + 입주예정연도
#                  경기 + 인천 + 혼합식 + 계단식 + 복도식 +
#                  경기_가평군 + 경기_고양시 + 경기_과천시 + 경기_광명시 + 경기_광주시 + 경기_구리시 + 경기_군포시 + 경기_김포시 + 경기_남양주시 + 경기_동두천시 + 경기_부천시 + 경기_성남시 + 경기_수원시 + 경기_시흥시 + 경기_안산시 + 경기_안성시 + 경기_안양시 + 경기_양주시 + 경기_양평군 + 경기_여주시 + 경기_연천군 + 경기_오산시 + 경기_용인시 + 경기_의왕시 + 경기_의정부시 + 경기_이천시 + 경기_파주시 + 경기_평택시 + 경기_포천시 + 경기_하남시 + 경기_화성시 + 서울_강남구 + 서울_강동구 + 서울_강북구 + 서울_강서구 + 서울_관악구 + 서울_광진구 + 서울_구로구 + 서울_노원구 + 서울_도봉구 + 서울_동대문구 + 서울_동작구 + 서울_서대문구 + 서울_서초구 + 서울_성동구 + 서울_성북구 + 서울_송파구 + 서울_양천구 + 서울_영등포구 + 서울_은평구 + 서울_종로구 + 서울_중구 + 서울_중랑구 + 인천_강화군 + 인천_계양구 + 인천_남동구 + 인천_동구 + 인천_미추홀구 + 인천_부평구 + 인천_서구 + 인천_연수구 + 인천_중구]

formula = '공급액_만원 ~ 세대수 + 임대세대수 + 최고층 + 최저층 + 총아파트동수 + 용적률 + 건폐율 + 세대평균_주차대수 + 공급면적 + 방수 + 욕실수 + 대형건설사_YES + 지하철역_거리_km + 병원수_1차 + 병원수_2차 + 병원수_3차 + 공원 + 대학 + 소매 + 음식 + 교육 + 장례식장 + 보건의료 + 유원지오락 + 총인구수  + 서울 + _419민주묘지역 + 가양역 + 가좌역 + 가평역 + 갈산역 + 강일역 + 거여역 + 건대입구역 + 걸포북변역 + 경기광주역 + 경인교대입구역 + 경전철의정부역 + 계양역 + 고덕역 + 고속터미널역 + 고진역 + 고촌역 + 광교역 + 광명사거리역 + 광명역 + 구래역 + 구리역 + 구의역 + 국제업무지구역 + 군포역 + 금곡역 + 금릉역 + 금정역 + 길동역 + 까치울역 + 남구로역 + 남부터미널역 + 남한산성입구역 + 녹천역 + 능곡역 + 달미역 + 당고개역 + 덕계역 + 덕소역 + 덕정역 + 도농역 + 도심역 + 도원역 + 독정역 + 돌곶이역 + 동두천중앙역 + 동수역 + 동오역 + 동인천역 + 동천역 + 동춘역 + 둔전역 + 둔촌오륜역 + 등촌역 + 디지털미디어시티역 + 마곡역 + 마석역 + 마전역 + 마천역 + 망우역 + 망포역 + 매교역 + 매탄권선역 + 면목역 + 명지대역 + 명학역 + 모란역 + 문산역 + 문학경기장역 + 미아역 + 박촌역 + 발산역 + 백운역 + 범계역 + 범골역 + 별내역 + 병점역 + 보평역 + 봉천역 + 부개역 + 부발역 + 부천역 + 부평역 + 산성역 + 삼가역 + 삼동역 + 삼성역 + 삼양사거리역 + 상갈역 + 상도역 + 상동역 + 상일동역 + 상현역 + 서동탄역 + 서울대벤처타운역 + 서울대입구역 + 서정리역 + 석남역 + 석바위시장역 + 선부역 + 성환역 + 세류역 + 세마역 + 센트럴파크역 + 소새울역 + 소요산역 + 송도달빛축제공원역 + 송정역 + 송탄역 + 수색역 + 숭의역 + 시민공원역 + 시흥시청역 + 신길역 + 신길온천역 + 신답역 + 신반포역 + 신설동역 + 신원역 + 신정네거리역 + 신천역 + 쌍문역 + 아시아드경기장역 + 안산역 + 안암역 + 안양역 + 야당역 + 야목역 + 양주역 + 양천향교역 + 양촌역 + 양평역 + 어정역 + 어천역 + 여주역 + 역곡역 + 연수역 + 영등포구청역 + 영종역 + 예술회관역 + 오목천역 + 오빈역 + 오산대역 + 오산역 + 완정역 + 왕길역 + 외대앞역 + 용마산역 + 용문역 + 운동장송담대역 + 운서역 + 운정역 + 원당역 + 을지로4가역 + 응암역 + 의정부시청역 + 의정부역 + 의정부중앙역 + 이천역 + 인덕원역 + 인천대입구역 + 인천시청역 + 인하대역 + 일원역 + 작전역 + 장한평역 + 정부과천청사역 + 정왕역 + 제물포역 + 주안역 + 죽전역 + 중동역 + 중앙역 + 중화역 + 지식정보단지역 + 지제역 + 지행역 + 진위역 + 진접역 + 창신역 + 천왕역 + 천호역 + 철산역 + 청라국제도시역 + 청량리역 + 청명역 + 초월역 + 탑석역 + 테크노파크역 + 퇴계원역 + 파주역 + 평촌역 + 평택역 + 하남풍산역 + 행신역 + 화곡역 + 화서역 + 화전역 + 흑석역 + 흥선역 +  가평군 + 강남구 + 강동구 + 강북구 + 강서구 + 강화군 + 계양구 + 고양시 + 과천시 + 관악구 + 광명시 + 광주시 + 광진구 + 구로구 + 구리시 + 군포시 + 김포시 + 남동구 + 남양주시 + 노원구 + 도봉구 + 동구 + 동대문구 + 동두천시 + 동작구 + 미추홀구 + 부천시 + 부평구 + 서구 + 서대문구 + 서초구 + 성남시 + 성동구 + 성북구 + 송파구 + 수원시 + 시흥시 + 안산시 + 안성시 + 안양시 + 양주시 + 양천구 + 양평군 + 여주시 + 연수구 + 연천군 + 영등포구 + 오산시 + 용인시 + 은평구 + 의왕시 + 의정부시 + 이천시 + 종로구 + 중구 + 중랑구 + 파주시 + 평택시 + 포천시 + 하남시 + 화성시'
results = smf.ols(formula, apt).fit()

print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                 공급액_만원   R-squared:                       0.922
Model:                            OLS   Adj. R-squared:                  0.914
Method:                 Least Squares   F-statistic:                     124.4
Date:                Sat, 30 Sep 2023   Prob (F-statistic):               0.00
Time:                        23:16:50   Log-Likelihood:                -29615.
No. Observations:                2828   AIC:                         5.972e+04
Df Residuals:                    2582   BIC:                         6.118e+04
Df Model:                         245                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept  -3.003e+04   2525.906    -11.887      0.0

## **3-2) 3차 회귀분석 VIF** 

In [84]:
# 회귀 모델에 사용할 독립 변수 선택
independent_vars = ['세대수','임대세대수','최고층','최저층','총아파트동수', '대형건설사_YES',
                    '용적률','건폐율','세대평균_주차대수','공급면적','방수','욕실수',
                    '지하철역_거리_km','병원수_1차','병원수_2차','병원수_3차','공원','대학',
                    '소매','음식','교육','장례식장','보건의료','유원지오락', '총인구수','서울',
                    '_419민주묘지역','가양역','가좌역','가평역','갈산역','강일역','거여역','건대입구역','걸포북변역','경기광주역','경인교대입구역','경전철의정부역','계양역',
                    '고덕역','고속터미널역','고진역','고촌역','광교역','광명사거리역','광명역','구래역','구리역','구의역','국제업무지구역','군포역','금릉역','금정역','길동역','까치울역',
                    '남구로역','남부터미널역','남한산성입구역','능곡역','달미역','당고개역','덕계역','덕소역','덕정역','도농역','도심역','도원역','독정역','돌곶이역',
                    '동두천중앙역','동수역','동오역','동인천역','동천역','동춘역','둔전역','둔촌오륜역','등촌역','디지털미디어시티역','마곡역','마전역','마천역','망우역','망포역',
                    '매교역','매탄권선역','면목역','명지대역','명학역','모란역','문산역','문학경기장역','미아역','박촌역','발산역','백운역','범계역','범골역','별내역','병점역',
                    '보평역','봉천역','부개역','부천역','부평역','산성역','삼가역','삼동역','삼성역','상갈역','상도역','상일동역','상현역','서동탄역','서울대벤처타운역','서정리역',
                    '석남역','석바위시장역','성환역','세마역','센트럴파크역','소새울역','소요산역','송도달빛축제공원역','송정역','송탄역','수색역','숭의역','시민공원역','시흥시청역',
                    '신길온천역','신답역','신반포역','신설동역','신원역','신정네거리역','신천역','쌍문역','아시아드경기장역','안산역','안양역','야당역','야목역','양주역','양천향교역','양촌역','양평역',
                    '어정역','어천역','여주역','역곡역','연수역','영등포구청역','영종역','오목천역','오빈역','오산대역','오산역','완정역','왕길역','외대앞역','용마산역','용문역',
                    '운동장송담대역','운서역','운정역','을지로4가역','응암역','의정부시청역','의정부역','의정부중앙역','이천역','인덕원역','인천대입구역','인하대역','일원역',
                    '작전역','장한평역','정왕역','제물포역','주안역','죽전역','중동역','중앙역','중화역','지식정보단지역','지제역','지행역','진위역','진접역','창신역','천호역','철산역',
                    '청라국제도시역','청량리역','청명역','초월역','테크노파크역','퇴계원역','파주역','평촌역','평택역','하남풍산역','행신역','화곡역','화서역','화전역','흑석역','흥선역',
                    '가평군','강남구','강동구','강북구','강서구','강화군','계양구','고양시','과천시','관악구','광명시','광주시','광진구','구로구','구리시','군포시','김포시',
                    '남양주시','노원구','도봉구','동구','동대문구','동두천시','동작구','미추홀구','부천시','부평구','서구','서대문구','중구','서초구','성남시','성동구','성북구',
                    '송파구','수원시','시흥시','안산시','안성시','안양시','양주시','양천구','양평군','여주시','연수구','연천군','영등포구','오산시','용인시','은평구',
                    '의왕시','의정부시','이천시','중구','종로구','중랑구','파주시','평택시','포천시','하남시','화성시']


# 회귀 모델 적합
X = sm.add_constant(apt[independent_vars])
y = apt['공급액_만원']
model = sm.OLS(y, X)
results = model.fit()

In [85]:
# GPT 코드 <VIF가 10 이상인 것만 뽑아내기>

high_vif_vars = []
high_vif_values = []
threshold = 10  #임계값 설정

for i in range(1, X.shape[1]):  # 첫 번째 컬럼은 상수항이므로 제외
    vif = variance_inflation_factor(X.values, i)
    if vif >= threshold:
        high_vif_vars.append(X.columns[i])
        high_vif_values.append(vif)

print("Variables with VIF >= {}:".format(threshold))
for var, vif in zip(high_vif_vars, high_vif_values):
    print("{}: {:.2f}".format(var, vif))

/Users/jang-yeji/miniconda3/envs/dataanalysis/lib/python3.8/site-packages/statsmodels/stats/outliers_influence.py:198: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)


Variables with VIF >= 10:
세대수: 44.06
임대세대수: 28.05
총아파트동수: 30.67
대학: 13.71
소매: inf
음식: inf
교육: inf
장례식장: inf
보건의료: inf
유원지오락: inf
총인구수: inf
서울: inf
가좌역: inf
가평역: 10.11
강일역: inf
거여역: inf
건대입구역: inf
걸포북변역: inf
경기광주역: 10.34
고덕역: inf
고속터미널역: inf
광명사거리역: inf
광명역: inf
구래역: inf
구리역: inf
구의역: inf
국제업무지구역: 31.55
군포역: inf
금릉역: inf
금정역: inf
길동역: inf
남부터미널역: inf
남한산성입구역: inf
당고개역: inf
덕계역: 19.95
덕정역: 13.07
동춘역: 25.44
둔촌오륜역: inf
등촌역: inf
디지털미디어시티역: inf
마곡역: inf
마천역: inf
망우역: inf
면목역: inf
명학역: inf
모란역: inf
문산역: inf
발산역: inf
범계역: inf
산성역: inf
삼성역: inf
상도역: inf
상일동역: inf
센트럴파크역: 10.59
소요산역: inf
송도달빛축제공원역: 21.55
송정역: inf
수색역: inf
시흥시청역: inf
신길온천역: inf
신답역: inf
신반포역: inf
신설동역: inf
신원역: inf
신정네거리역: inf
신천역: inf
안양역: inf
야당역: inf
양촌역: inf
양평역: inf
여주역: 10.08
영종역: 36.35
오빈역: inf
외대앞역: inf
용마산역: inf
용문역: inf
운서역: 36.61
운정역: inf
을지로4가역: inf
응암역: inf
인천대입구역: 64.09
일원역: inf
장한평역: inf
정왕역: inf
중화역: inf
창신역: inf
천호역: inf
철산역: inf
청량리역: inf
초월역: 17.56
파주역: inf
평촌역: inf
평택역: 11.52
하남풍산역: inf
화곡역: inf
흑석역: inf
가평군: 

## **4-1) 4차 회귀분석** 

In [86]:
# 4차 ols표 

import statsmodels.formula. api as smf

# 포함하지 않을 X = ['아파트명','법정동주소','지하철역','역사명','노선명_리스트','기초','광역기초' 
#                  위도 + 경도 + 최대공급면적 + 최소공급면적 + 전용면적 + 전용율 + 입주예정연도
#                  경기 + 인천 + 혼합식 + 계단식 + 복도식 +
#                  경기_가평군 + 경기_고양시 + 경기_과천시 + 경기_광명시 + 경기_광주시 + 경기_구리시 + 경기_군포시 + 경기_김포시 + 경기_남양주시 + 경기_동두천시 + 경기_부천시 + 경기_성남시 + 경기_수원시 + 경기_시흥시 + 경기_안산시 + 경기_안성시 + 경기_안양시 + 경기_양주시 + 경기_양평군 + 경기_여주시 + 경기_연천군 + 경기_오산시 + 경기_용인시 + 경기_의왕시 + 경기_의정부시 + 경기_이천시 + 경기_파주시 + 경기_평택시 + 경기_포천시 + 경기_하남시 + 경기_화성시 + 서울_강남구 + 서울_강동구 + 서울_강북구 + 서울_강서구 + 서울_관악구 + 서울_광진구 + 서울_구로구 + 서울_노원구 + 서울_도봉구 + 서울_동대문구 + 서울_동작구 + 서울_서대문구 + 서울_서초구 + 서울_성동구 + 서울_성북구 + 서울_송파구 + 서울_양천구 + 서울_영등포구 + 서울_은평구 + 서울_종로구 + 서울_중구 + 서울_중랑구 + 인천_강화군 + 인천_계양구 + 인천_남동구 + 인천_동구 + 인천_미추홀구 + 인천_부평구 + 인천_서구 + 인천_연수구 + 인천_중구
#                  _419민주묘지역 + 가양역 + 가좌역 + 가평역 + 갈산역 + 강일역 + 거여역 + 건대입구역 + 걸포북변역 + 경기광주역 + 경인교대입구역 + 경전철의정부역 + 계양역 + 고덕역 + 고속터미널역 + 고진역 + 고촌역 + 광교역 + 광명사거리역 + 광명역 + 구래역 + 구리역 + 구의역 + 국제업무지구역 + 군포역 + 금곡역 + 금릉역 + 금정역 + 길동역 + 까치울역 + 남구로역 + 남부터미널역 + 남한산성입구역 + 녹천역 + 능곡역 + 달미역 + 당고개역 + 덕계역 + 덕소역 + 덕정역 + 도농역 + 도심역 + 도원역 + 독정역 + 돌곶이역 + 동두천중앙역 + 동수역 + 동오역 + 동인천역 + 동천역 + 동춘역 + 둔전역 + 둔촌오륜역 + 등촌역 + 디지털미디어시티역 + 마곡역 + 마석역 + 마전역 + 마천역 + 망우역 + 망포역 + 매교역 + 매탄권선역 + 면목역 + 명지대역 + 명학역 + 모란역 + 문산역 + 문학경기장역 + 미아역 + 박촌역 + 발산역 + 백운역 + 범계역 + 범골역 + 별내역 + 병점역 + 보평역 + 봉천역 + 부개역 + 부발역 + 부천역 + 부평역 + 산성역 + 삼가역 + 삼동역 + 삼성역 + 삼양사거리역 + 상갈역 + 상도역 + 상동역 + 상일동역 + 상현역 + 서동탄역 + 서울대벤처타운역 + 서울대입구역 + 서정리역 + 석남역 + 석바위시장역 + 선부역 + 성환역 + 세류역 + 세마역 + 센트럴파크역 + 소새울역 + 소요산역 + 송도달빛축제공원역 + 송정역 + 송탄역 + 수색역 + 숭의역 + 시민공원역 + 시흥시청역 + 신길역 + 신길온천역 + 신답역 + 신반포역 + 신설동역 + 신원역 + 신정네거리역 + 신천역 + 쌍문역 + 아시아드경기장역 + 안산역 + 안암역 + 안양역 + 야당역 + 야목역 + 양주역 + 양천향교역 + 양촌역 + 양평역 + 어정역 + 어천역 + 여주역 + 역곡역 + 연수역 + 영등포구청역 + 영종역 + 예술회관역 + 오목천역 + 오빈역 + 오산대역 + 오산역 + 완정역 + 왕길역 + 외대앞역 + 용마산역 + 용문역 + 운동장송담대역 + 운서역 + 운정역 + 원당역 + 을지로4가역 + 응암역 + 의정부시청역 + 의정부역 + 의정부중앙역 + 이천역 + 인덕원역 + 인천대입구역 + 인천시청역 + 인하대역 + 일원역 + 작전역 + 장한평역 + 정부과천청사역 + 정왕역 + 제물포역 + 주안역 + 죽전역 + 중동역 + 중앙역 + 중화역 + 지식정보단지역 + 지제역 + 지행역 + 진위역 + 진접역 + 창신역 + 천왕역 + 천호역 + 철산역 + 청라국제도시역 + 청량리역 + 청명역 + 초월역 + 탑석역 + 테크노파크역 + 퇴계원역 + 파주역 + 평촌역 + 평택역 + 하남풍산역 + 행신역 + 화곡역 + 화서역 + 화전역 + 흑석역 + 흥선역]

formula = '공급액_만원 ~ 세대수 + 임대세대수 + 최고층 + 최저층 + 총아파트동수 + 용적률 + 건폐율 + 세대평균_주차대수 + 공급면적 + 방수 + 욕실수 + 대형건설사_YES + 지하철역_거리_km + 병원수_1차 + 병원수_2차 + 병원수_3차 + 공원 + 대학 + 소매 + 음식 + 교육 + 장례식장 + 보건의료 + 유원지오락 + 총인구수  + 서울 +  가평군 + 강남구 + 강동구 + 강북구 + 강서구 + 강화군 + 계양구 + 고양시 + 과천시 + 관악구 + 광명시 + 광주시 + 광진구 + 구로구 + 구리시 + 군포시 + 김포시 + 남동구 + 남양주시 + 노원구 + 도봉구 + 동구 + 동대문구 + 동두천시 + 동작구 + 미추홀구 + 부천시 + 부평구 + 서구 + 서대문구 + 서초구 + 성남시 + 성동구 + 성북구 + 송파구 + 수원시 + 시흥시 + 안산시 + 안성시 + 안양시 + 양주시 + 양천구 + 양평군 + 여주시 + 연수구 + 연천군 + 영등포구 + 오산시 + 용인시 + 은평구 + 의왕시 + 의정부시 + 이천시 + 종로구 + 중구 + 중랑구 + 파주시 + 평택시 + 포천시 + 하남시 + 화성시'
results = smf.ols(formula, apt).fit()

print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                 공급액_만원   R-squared:                       0.864
Model:                            OLS   Adj. R-squared:                  0.860
Method:                 Least Squares   F-statistic:                     221.2
Date:                Sat, 30 Sep 2023   Prob (F-statistic):               0.00
Time:                        23:17:15   Log-Likelihood:                -30398.
No. Observations:                2828   AIC:                         6.096e+04
Df Residuals:                    2748   BIC:                         6.143e+04
Df Model:                          79                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept   -3.65e+04   2370.629    -15.397      0.0

In [87]:
output_filename = 'regression_result.txt'
with open(output_filename, 'w') as f:
    f.write(results.summary().as_text()) 

## **4-2) 4차 회귀분석 VIF** 

In [88]:
# 회귀 모델에 사용할 독립 변수 선택
independent_vars = ['세대수','임대세대수','최고층','최저층','총아파트동수', '대형건설사_YES',
                    '용적률','건폐율','세대평균_주차대수','공급면적','방수','욕실수',
                    '지하철역_거리_km','병원수_1차','병원수_2차','병원수_3차','공원','대학',
                    '소매','음식','교육','장례식장','보건의료','유원지오락', '총인구수','서울',
                    '가평군','강남구','강동구','강북구','강서구','강화군','계양구','고양시','과천시','관악구','광명시','광주시','광진구','구로구','구리시','군포시','김포시',
                    '남양주시','노원구','도봉구','동구','동대문구','동두천시','동작구','미추홀구','부천시','부평구','서구','서대문구','중구','서초구','성남시','성동구','성북구',
                    '송파구','수원시','시흥시','안산시','안성시','안양시','양주시','양천구','양평군','여주시','연수구','연천군','영등포구','오산시','용인시','은평구',
                    '의왕시','의정부시','이천시','중구','종로구','중랑구','파주시','평택시','포천시','하남시','화성시']


# 회귀 모델 적합
X = sm.add_constant(apt[independent_vars])
y = apt['공급액_만원']
model = sm.OLS(y, X)
results = model.fit()

In [89]:
# GPT 코드 <VIF가 10 이상인 것만 뽑아내기>

high_vif_vars = []
high_vif_values = []
threshold = 10  #임계값 설정

for i in range(1, X.shape[1]):  # 첫 번째 컬럼은 상수항이므로 제외
    vif = variance_inflation_factor(X.values, i)
    if vif >= threshold:
        high_vif_vars.append(X.columns[i])
        high_vif_values.append(vif)

print("Variables with VIF >= {}:".format(threshold))
for var, vif in zip(high_vif_vars, high_vif_values):
    print("{}: {:.2f}".format(var, vif))

/Users/jang-yeji/miniconda3/envs/dataanalysis/lib/python3.8/site-packages/statsmodels/stats/outliers_influence.py:198: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)


Variables with VIF >= 10:
세대수: 13.93
총아파트동수: 12.12
소매: inf
음식: inf
교육: inf
장례식장: inf
보건의료: inf
유원지오락: inf
총인구수: inf
서울: inf
가평군: inf
강남구: inf
강동구: inf
강북구: inf
강서구: inf
강화군: 9007199254740992.00
계양구: inf
고양시: inf
과천시: inf
관악구: inf
광명시: inf
광주시: inf
광진구: inf
구로구: inf
구리시: inf
군포시: inf
김포시: inf
남양주시: inf
노원구: inf
도봉구: inf
동구: inf
동대문구: inf
동두천시: inf
동작구: inf
미추홀구: 9007199254740992.00
부천시: inf
부평구: inf
서구: inf
서대문구: inf
중구: inf
서초구: inf
성남시: inf
성동구: inf
성북구: inf
송파구: inf
수원시: inf
시흥시: inf
안산시: inf
안성시: inf
안양시: inf
양주시: inf
양천구: inf
양평군: inf
여주시: inf
연수구: inf
연천군: inf
영등포구: inf
오산시: inf
용인시: inf
은평구: inf
의왕시: inf
의정부시: inf
이천시: inf
중구: inf
종로구: inf
중랑구: inf
파주시: inf
평택시: inf
포천시: inf
하남시: inf
화성시: inf


## **5-1) 5차 회귀분석** 

In [90]:
# 5차 ols표 

import statsmodels.formula. api as smf

# 포함하지 않을 X = ['아파트명','법정동주소','지하철역','역사명','노선명_리스트','기초','광역기초' 
#                  위도 + 경도 + 최대공급면적 + 최소공급면적 + 전용면적 + 전용율 + 입주예정연도
#                 서울 + 경기 + 인천 + 혼합식 + 계단식 + 복도식 +
#                 가평군 + 강남구 + 강동구 + 강북구 + 강서구 + 강화군 + 계양구 + 고양시 + 과천시 + 관악구 + 광명시 + 광주시 + 광진구 + 구로구 + 구리시 + 군포시 + 김포시 + 남동구 + 남양주시 + 노원구 + 도봉구 + 동구 + 동대문구 + 동두천시 + 동작구 + 미추홀구 + 부천시 + 부평구 + 서구 + 서대문구 + 서초구 + 성남시 + 성동구 + 성북구 + 송파구 + 수원시 + 시흥시 + 안산시 + 안성시 + 안양시 + 양주시 + 양천구 + 양평군 + 여주시 + 연수구 + 연천군 + 영등포구 + 오산시 + 용인시 + 은평구 + 의왕시 + 의정부시 + 이천시 + 종로구 + 중구 + 중랑구 + 파주시 + 평택시 + 포천시 + 하남시 + 화성시
#                  _419민주묘지역 + 가양역 + 가좌역 + 가평역 + 갈산역 + 강일역 + 거여역 + 건대입구역 + 걸포북변역 + 경기광주역 + 경인교대입구역 + 경전철의정부역 + 계양역 + 고덕역 + 고속터미널역 + 고진역 + 고촌역 + 광교역 + 광명사거리역 + 광명역 + 구래역 + 구리역 + 구의역 + 국제업무지구역 + 군포역 + 금곡역 + 금릉역 + 금정역 + 길동역 + 까치울역 + 남구로역 + 남부터미널역 + 남한산성입구역 + 녹천역 + 능곡역 + 달미역 + 당고개역 + 덕계역 + 덕소역 + 덕정역 + 도농역 + 도심역 + 도원역 + 독정역 + 돌곶이역 + 동두천중앙역 + 동수역 + 동오역 + 동인천역 + 동천역 + 동춘역 + 둔전역 + 둔촌오륜역 + 등촌역 + 디지털미디어시티역 + 마곡역 + 마석역 + 마전역 + 마천역 + 망우역 + 망포역 + 매교역 + 매탄권선역 + 면목역 + 명지대역 + 명학역 + 모란역 + 문산역 + 문학경기장역 + 미아역 + 박촌역 + 발산역 + 백운역 + 범계역 + 범골역 + 별내역 + 병점역 + 보평역 + 봉천역 + 부개역 + 부발역 + 부천역 + 부평역 + 산성역 + 삼가역 + 삼동역 + 삼성역 + 삼양사거리역 + 상갈역 + 상도역 + 상동역 + 상일동역 + 상현역 + 서동탄역 + 서울대벤처타운역 + 서울대입구역 + 서정리역 + 석남역 + 석바위시장역 + 선부역 + 성환역 + 세류역 + 세마역 + 센트럴파크역 + 소새울역 + 소요산역 + 송도달빛축제공원역 + 송정역 + 송탄역 + 수색역 + 숭의역 + 시민공원역 + 시흥시청역 + 신길역 + 신길온천역 + 신답역 + 신반포역 + 신설동역 + 신원역 + 신정네거리역 + 신천역 + 쌍문역 + 아시아드경기장역 + 안산역 + 안암역 + 안양역 + 야당역 + 야목역 + 양주역 + 양천향교역 + 양촌역 + 양평역 + 어정역 + 어천역 + 여주역 + 역곡역 + 연수역 + 영등포구청역 + 영종역 + 예술회관역 + 오목천역 + 오빈역 + 오산대역 + 오산역 + 완정역 + 왕길역 + 외대앞역 + 용마산역 + 용문역 + 운동장송담대역 + 운서역 + 운정역 + 원당역 + 을지로4가역 + 응암역 + 의정부시청역 + 의정부역 + 의정부중앙역 + 이천역 + 인덕원역 + 인천대입구역 + 인천시청역 + 인하대역 + 일원역 + 작전역 + 장한평역 + 정부과천청사역 + 정왕역 + 제물포역 + 주안역 + 죽전역 + 중동역 + 중앙역 + 중화역 + 지식정보단지역 + 지제역 + 지행역 + 진위역 + 진접역 + 창신역 + 천왕역 + 천호역 + 철산역 + 청라국제도시역 + 청량리역 + 청명역 + 초월역 + 탑석역 + 테크노파크역 + 퇴계원역 + 파주역 + 평촌역 + 평택역 + 하남풍산역 + 행신역 + 화곡역 + 화서역 + 화전역 + 흑석역 + 흥선역]

formula = '공급액_만원 ~ 세대수 + 임대세대수 + 최고층 + 최저층 + 총아파트동수 + 용적률 + 건폐율 + 세대평균_주차대수 + 공급면적 + 방수 + 욕실수 + 대형건설사_YES + 지하철역_거리_km + 병원수_1차 + 병원수_2차 + 병원수_3차 + 공원 + 대학 + 소매 + 음식 + 교육 + 장례식장 + 보건의료 + 유원지오락 + 총인구수  + 경기_가평군 + 경기_고양시 + 경기_과천시 + 경기_광명시 + 경기_광주시 + 경기_구리시 + 경기_군포시 + 경기_김포시 + 경기_남양주시 + 경기_동두천시 + 경기_부천시 + 경기_성남시 + 경기_수원시 + 경기_시흥시 + 경기_안산시 + 경기_안성시 + 경기_안양시 + 경기_양주시 + 경기_양평군 + 경기_여주시 + 경기_연천군 + 경기_오산시 + 경기_용인시 + 경기_의왕시 + 경기_의정부시 + 경기_이천시 + 경기_파주시 + 경기_평택시 + 경기_포천시 + 경기_하남시 + 경기_화성시 + 서울_강남구 + 서울_강동구 + 서울_강북구 + 서울_강서구 + 서울_관악구 + 서울_광진구 + 서울_구로구 + 서울_노원구 + 서울_도봉구 + 서울_동대문구 + 서울_동작구 + 서울_서대문구 + 서울_서초구 + 서울_성동구 + 서울_성북구 + 서울_송파구 + 서울_양천구 + 서울_영등포구 + 서울_은평구 + 서울_종로구 + 서울_중구 + 서울_중랑구 + 인천_강화군 + 인천_계양구 + 인천_남동구 + 인천_동구 + 인천_미추홀구 + 인천_부평구 + 인천_서구 + 인천_연수구 + 인천_중구'
results = smf.ols(formula, apt).fit()

print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                 공급액_만원   R-squared:                       0.864
Model:                            OLS   Adj. R-squared:                  0.860
Method:                 Least Squares   F-statistic:                     221.2
Date:                Sat, 30 Sep 2023   Prob (F-statistic):               0.00
Time:                        23:17:17   Log-Likelihood:                -30398.
No. Observations:                2828   AIC:                         6.096e+04
Df Residuals:                    2748   BIC:                         6.143e+04
Df Model:                          79                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept  -2.609e+04   2161.188    -12.070      0.0

In [91]:
output_filename = 'regression_result.txt'
with open(output_filename, 'w') as f:
    f.write(results.summary().as_text()) 

## **5-2) 5차 회귀분석 VIF** 

In [92]:
# 회귀 모델에 사용할 독립 변수 선택
independent_vars = ['세대수','임대세대수','최고층','최저층','총아파트동수', '대형건설사_YES',
                    '용적률','건폐율','세대평균_주차대수','공급면적','방수','욕실수',
                    '지하철역_거리_km','병원수_1차','병원수_2차','병원수_3차','공원','대학',
                    '소매','음식','교육','장례식장','보건의료','유원지오락', '총인구수',
                    '경기_고양시','경기_과천시','경기_광명시','경기_광주시','경기_구리시','경기_군포시','경기_김포시','경기_남양주시','경기_동두천시','경기_부천시','경기_성남시','경기_수원시',
                    '경기_시흥시','경기_안산시','경기_안성시','경기_안양시','경기_양주시','경기_양평군','경기_여주시','경기_연천군','경기_오산시','경기_용인시','경기_의왕시','경기_의정부시',
                    '경기_이천시','경기_파주시','경기_평택시','경기_포천시','경기_하남시','경기_화성시','서울_강남구','서울_강동구','서울_강북구','서울_강서구','서울_관악구','서울_광진구','서울_구로구',
                    '서울_노원구','서울_도봉구','서울_동대문구','서울_동작구','서울_서대문구','서울_서초구','서울_성동구','서울_성북구','서울_송파구','서울_양천구','서울_영등포구','서울_은평구',
                    '서울_종로구','서울_중구','서울_중랑구','인천_강화군','인천_계양구','인천_남동구','인천_동구','인천_미추홀구','인천_부평구','인천_서구','인천_연수구','인천_중구']


# 회귀 모델 적합
X = sm.add_constant(apt[independent_vars])
y = apt['공급액_만원']
model = sm.OLS(y, X)
results = model.fit()

In [93]:
# GPT 코드 <VIF가 10 이상인 것만 뽑아내기>

high_vif_vars = []
high_vif_values = []
threshold = 10  #임계값 설정

for i in range(1, X.shape[1]):  # 첫 번째 컬럼은 상수항이므로 제외
    vif = variance_inflation_factor(X.values, i)
    if vif >= threshold:
        high_vif_vars.append(X.columns[i])
        high_vif_values.append(vif)

print("Variables with VIF >= {}:".format(threshold))
for var, vif in zip(high_vif_vars, high_vif_values):
    print("{}: {:.2f}".format(var, vif))

/Users/jang-yeji/miniconda3/envs/dataanalysis/lib/python3.8/site-packages/statsmodels/stats/outliers_influence.py:198: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)


Variables with VIF >= 10:
세대수: 13.93
총아파트동수: 12.12
소매: inf
음식: inf
교육: inf
장례식장: inf
보건의료: inf
유원지오락: inf
총인구수: inf
경기_고양시: inf
경기_과천시: inf
경기_광명시: inf
경기_광주시: inf
경기_구리시: inf
경기_군포시: inf
경기_김포시: inf
경기_남양주시: inf
경기_동두천시: inf
경기_부천시: inf
경기_성남시: inf
경기_수원시: inf
경기_시흥시: inf
경기_안산시: inf
경기_안성시: inf
경기_안양시: inf
경기_양주시: inf
경기_양평군: inf
경기_여주시: inf
경기_연천군: 4503599627370496.00
경기_오산시: inf
경기_용인시: inf
경기_의왕시: inf
경기_의정부시: inf
경기_이천시: inf
경기_파주시: inf
경기_평택시: inf
경기_포천시: inf
경기_하남시: inf
경기_화성시: inf
서울_강남구: inf
서울_강동구: inf
서울_강북구: inf
서울_강서구: inf
서울_관악구: inf
서울_광진구: inf
서울_구로구: inf
서울_노원구: inf
서울_도봉구: inf
서울_동대문구: inf
서울_동작구: inf
서울_서대문구: inf
서울_서초구: inf
서울_성동구: inf
서울_성북구: inf
서울_송파구: inf
서울_양천구: inf
서울_영등포구: inf
서울_은평구: inf
서울_종로구: inf
서울_중구: inf
서울_중랑구: inf
인천_강화군: inf
인천_계양구: inf
인천_남동구: inf
인천_동구: inf
인천_미추홀구: inf
인천_부평구: inf
인천_서구: inf
인천_연수구: inf
인천_중구: inf


## **6-1) 6차 회귀분석** 

In [94]:
# 6차 ols표 

import statsmodels.formula. api as smf

# 포함하지 않을 X = ['아파트명','법정동주소','지하철역','역사명','노선명_리스트','기초','광역기초', '임대세대수' 
#                  위도 + 경도 + 최대공급면적 + 최소공급면적 + 전용면적 + 전용율 + 입주예정연도
#                 서울 + 경기 + 인천 + 혼합식 + 계단식 + 복도식 +
#                 가평군 + 강남구 + 강동구 + 강북구 + 강서구 + 강화군 + 계양구 + 고양시 + 과천시 + 관악구 + 광명시 + 광주시 + 광진구 + 구로구 + 구리시 + 군포시 + 김포시 + 남동구 + 남양주시 + 노원구 + 도봉구 + 동구 + 동대문구 + 동두천시 + 동작구 + 미추홀구 + 부천시 + 부평구 + 서구 + 서대문구 + 서초구 + 성남시 + 성동구 + 성북구 + 송파구 + 수원시 + 시흥시 + 안산시 + 안성시 + 안양시 + 양주시 + 양천구 + 양평군 + 여주시 + 연수구 + 연천군 + 영등포구 + 오산시 + 용인시 + 은평구 + 의왕시 + 의정부시 + 이천시 + 종로구 + 중구 + 중랑구 + 파주시 + 평택시 + 포천시 + 하남시 + 화성시
#                  _419민주묘지역 + 가양역 + 가좌역 + 가평역 + 갈산역 + 강일역 + 거여역 + 건대입구역 + 걸포북변역 + 경기광주역 + 경인교대입구역 + 경전철의정부역 + 계양역 + 고덕역 + 고속터미널역 + 고진역 + 고촌역 + 광교역 + 광명사거리역 + 광명역 + 구래역 + 구리역 + 구의역 + 국제업무지구역 + 군포역 + 금곡역 + 금릉역 + 금정역 + 길동역 + 까치울역 + 남구로역 + 남부터미널역 + 남한산성입구역 + 녹천역 + 능곡역 + 달미역 + 당고개역 + 덕계역 + 덕소역 + 덕정역 + 도농역 + 도심역 + 도원역 + 독정역 + 돌곶이역 + 동두천중앙역 + 동수역 + 동오역 + 동인천역 + 동천역 + 동춘역 + 둔전역 + 둔촌오륜역 + 등촌역 + 디지털미디어시티역 + 마곡역 + 마석역 + 마전역 + 마천역 + 망우역 + 망포역 + 매교역 + 매탄권선역 + 면목역 + 명지대역 + 명학역 + 모란역 + 문산역 + 문학경기장역 + 미아역 + 박촌역 + 발산역 + 백운역 + 범계역 + 범골역 + 별내역 + 병점역 + 보평역 + 봉천역 + 부개역 + 부발역 + 부천역 + 부평역 + 산성역 + 삼가역 + 삼동역 + 삼성역 + 삼양사거리역 + 상갈역 + 상도역 + 상동역 + 상일동역 + 상현역 + 서동탄역 + 서울대벤처타운역 + 서울대입구역 + 서정리역 + 석남역 + 석바위시장역 + 선부역 + 성환역 + 세류역 + 세마역 + 센트럴파크역 + 소새울역 + 소요산역 + 송도달빛축제공원역 + 송정역 + 송탄역 + 수색역 + 숭의역 + 시민공원역 + 시흥시청역 + 신길역 + 신길온천역 + 신답역 + 신반포역 + 신설동역 + 신원역 + 신정네거리역 + 신천역 + 쌍문역 + 아시아드경기장역 + 안산역 + 안암역 + 안양역 + 야당역 + 야목역 + 양주역 + 양천향교역 + 양촌역 + 양평역 + 어정역 + 어천역 + 여주역 + 역곡역 + 연수역 + 영등포구청역 + 영종역 + 예술회관역 + 오목천역 + 오빈역 + 오산대역 + 오산역 + 완정역 + 왕길역 + 외대앞역 + 용마산역 + 용문역 + 운동장송담대역 + 운서역 + 운정역 + 원당역 + 을지로4가역 + 응암역 + 의정부시청역 + 의정부역 + 의정부중앙역 + 이천역 + 인덕원역 + 인천대입구역 + 인천시청역 + 인하대역 + 일원역 + 작전역 + 장한평역 + 정부과천청사역 + 정왕역 + 제물포역 + 주안역 + 죽전역 + 중동역 + 중앙역 + 중화역 + 지식정보단지역 + 지제역 + 지행역 + 진위역 + 진접역 + 창신역 + 천왕역 + 천호역 + 철산역 + 청라국제도시역 + 청량리역 + 청명역 + 초월역 + 탑석역 + 테크노파크역 + 퇴계원역 + 파주역 + 평촌역 + 평택역 + 하남풍산역 + 행신역 + 화곡역 + 화서역 + 화전역 + 흑석역 + 흥선역
#                  경기_가평군 + 경기_고양시 + 경기_과천시 + 경기_광명시 + 경기_광주시 + 경기_구리시 + 경기_군포시 + 경기_김포시 + 경기_남양주시 + 경기_동두천시 + 경기_부천시 + 경기_성남시 + 경기_수원시 + 경기_시흥시 + 경기_안산시 + 경기_안성시 + 경기_안양시 + 경기_양주시 + 경기_양평군 + 경기_여주시 + 경기_연천군 + 경기_오산시 + 경기_용인시 + 경기_의왕시 + 경기_의정부시 + 경기_이천시 + 경기_파주시 + 경기_평택시 + 경기_포천시 + 경기_하남시 + 경기_화성시 + 서울_강남구 + 서울_강동구 + 서울_강북구 + 서울_강서구 + 서울_관악구 + 서울_광진구 + 서울_구로구 + 서울_노원구 + 서울_도봉구 + 서울_동대문구 + 서울_동작구 + 서울_서대문구 + 서울_서초구 + 서울_성동구 + 서울_성북구 + 서울_송파구 + 서울_양천구 + 서울_영등포구 + 서울_은평구 + 서울_종로구 + 서울_중구 + 서울_중랑구 + 인천_강화군 + 인천_계양구 + 인천_남동구 + 인천_동구 + 인천_미추홀구 + 인천_부평구 + 인천_서구 + 인천_연수구 + 인천_중구']

formula = '공급액_만원 ~ 세대수 + 최고층 + 최저층 + 총아파트동수 + 용적률 + 건폐율 + 세대평균_주차대수 + 공급면적 + 방수 + 욕실수 + 대형건설사_YES + 지하철역_거리_km + 병원수_1차 + 병원수_2차 + 병원수_3차 + 공원 + 대학 + 소매 + 음식 + 교육 + 장례식장 + 보건의료 + 유원지오락 + 총인구수 + 서울'
results = smf.ols(formula, apt).fit()

print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                 공급액_만원   R-squared:                       0.775
Model:                            OLS   Adj. R-squared:                  0.773
Method:                 Least Squares   F-statistic:                     385.8
Date:                Sat, 30 Sep 2023   Prob (F-statistic):               0.00
Time:                        23:17:18   Log-Likelihood:                -31112.
No. Observations:                2828   AIC:                         6.228e+04
Df Residuals:                    2802   BIC:                         6.243e+04
Df Model:                          25                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept  -3.151e+04   2640.216    -11.936      0.0

## **6-2) 6차 회귀분석 VIF** 

In [95]:
# 회귀 모델에 사용할 독립 변수 선택
independent_vars = ['세대수','최고층','최저층','총아파트동수', '대형건설사_YES',
                    '용적률','건폐율','세대평균_주차대수','공급면적','방수','욕실수',
                    '지하철역_거리_km','병원수_1차','병원수_2차','병원수_3차','공원','대학',
                    '소매','음식','교육','장례식장','보건의료','유원지오락', '총인구수', '서울']


# 회귀 모델 적합
X = sm.add_constant(apt[independent_vars])
y = apt['공급액_만원']
model = sm.OLS(y, X)
results = model.fit()

In [96]:
# GPT 코드 <VIF가 10 이상인 것만 뽑아내기>

high_vif_vars = []
high_vif_values = []
threshold = 10  #임계값 설정

for i in range(1, X.shape[1]):  # 첫 번째 컬럼은 상수항이므로 제외
    vif = variance_inflation_factor(X.values, i)
    if vif >= threshold:
        high_vif_vars.append(X.columns[i])
        high_vif_values.append(vif)

print("Variables with VIF >= {}:".format(threshold))
for var, vif in zip(high_vif_vars, high_vif_values):
    print("{}: {:.2f}".format(var, vif))

Variables with VIF >= 10:
소매: 29.99
음식: 37.56
교육: 23.98
유원지오락: 21.12
총인구수: 14.04


## **7-1) 7차 회귀분석** 

In [97]:
# 7차 ols표 

import statsmodels.formula. api as smf

# 포함하지 않을 X = ['아파트명','법정동주소','지하철역','역사명','노선명_리스트','기초','광역기초', '임대세대수' 
#                 위도 + 경도 + 최대공급면적 + 최소공급면적 + 전용면적 + 전용율 + 입주예정연도
#                 서울 + 경기 + 인천 + 혼합식 + 계단식 + 복도식 +
#                 가평군 + 강남구 + 강동구 + 강북구 + 강서구 + 강화군 + 계양구 + 고양시 + 과천시 + 관악구 + 광명시 + 광주시 + 광진구 + 구로구 + 구리시 + 군포시 + 김포시 + 남동구 + 남양주시 + 노원구 + 도봉구 + 동구 + 동대문구 + 동두천시 + 동작구 + 미추홀구 + 부천시 + 부평구 + 서구 + 서대문구 + 서초구 + 성남시 + 성동구 + 성북구 + 송파구 + 수원시 + 시흥시 + 안산시 + 안성시 + 안양시 + 양주시 + 양천구 + 양평군 + 여주시 + 연수구 + 연천군 + 영등포구 + 오산시 + 용인시 + 은평구 + 의왕시 + 의정부시 + 이천시 + 종로구 + 중구 + 중랑구 + 파주시 + 평택시 + 포천시 + 하남시 + 화성시
#                 _419민주묘지역 + 가양역 + 가좌역 + 가평역 + 갈산역 + 강일역 + 거여역 + 건대입구역 + 걸포북변역 + 경기광주역 + 경인교대입구역 + 경전철의정부역 + 계양역 + 고덕역 + 고속터미널역 + 고진역 + 고촌역 + 광교역 + 광명사거리역 + 광명역 + 구래역 + 구리역 + 구의역 + 국제업무지구역 + 군포역 + 금곡역 + 금릉역 + 금정역 + 길동역 + 까치울역 + 남구로역 + 남부터미널역 + 남한산성입구역 + 녹천역 + 능곡역 + 달미역 + 당고개역 + 덕계역 + 덕소역 + 덕정역 + 도농역 + 도심역 + 도원역 + 독정역 + 돌곶이역 + 동두천중앙역 + 동수역 + 동오역 + 동인천역 + 동천역 + 동춘역 + 둔전역 + 둔촌오륜역 + 등촌역 + 디지털미디어시티역 + 마곡역 + 마석역 + 마전역 + 마천역 + 망우역 + 망포역 + 매교역 + 매탄권선역 + 면목역 + 명지대역 + 명학역 + 모란역 + 문산역 + 문학경기장역 + 미아역 + 박촌역 + 발산역 + 백운역 + 범계역 + 범골역 + 별내역 + 병점역 + 보평역 + 봉천역 + 부개역 + 부발역 + 부천역 + 부평역 + 산성역 + 삼가역 + 삼동역 + 삼성역 + 삼양사거리역 + 상갈역 + 상도역 + 상동역 + 상일동역 + 상현역 + 서동탄역 + 서울대벤처타운역 + 서울대입구역 + 서정리역 + 석남역 + 석바위시장역 + 선부역 + 성환역 + 세류역 + 세마역 + 센트럴파크역 + 소새울역 + 소요산역 + 송도달빛축제공원역 + 송정역 + 송탄역 + 수색역 + 숭의역 + 시민공원역 + 시흥시청역 + 신길역 + 신길온천역 + 신답역 + 신반포역 + 신설동역 + 신원역 + 신정네거리역 + 신천역 + 쌍문역 + 아시아드경기장역 + 안산역 + 안암역 + 안양역 + 야당역 + 야목역 + 양주역 + 양천향교역 + 양촌역 + 양평역 + 어정역 + 어천역 + 여주역 + 역곡역 + 연수역 + 영등포구청역 + 영종역 + 예술회관역 + 오목천역 + 오빈역 + 오산대역 + 오산역 + 완정역 + 왕길역 + 외대앞역 + 용마산역 + 용문역 + 운동장송담대역 + 운서역 + 운정역 + 원당역 + 을지로4가역 + 응암역 + 의정부시청역 + 의정부역 + 의정부중앙역 + 이천역 + 인덕원역 + 인천대입구역 + 인천시청역 + 인하대역 + 일원역 + 작전역 + 장한평역 + 정부과천청사역 + 정왕역 + 제물포역 + 주안역 + 죽전역 + 중동역 + 중앙역 + 중화역 + 지식정보단지역 + 지제역 + 지행역 + 진위역 + 진접역 + 창신역 + 천왕역 + 천호역 + 철산역 + 청라국제도시역 + 청량리역 + 청명역 + 초월역 + 탑석역 + 테크노파크역 + 퇴계원역 + 파주역 + 평촌역 + 평택역 + 하남풍산역 + 행신역 + 화곡역 + 화서역 + 화전역 + 흑석역 + 흥선역
#                 경기_가평군 + 경기_고양시 + 경기_과천시 + 경기_광명시 + 경기_광주시 + 경기_구리시 + 경기_군포시 + 경기_김포시 + 경기_남양주시 + 경기_동두천시 + 경기_부천시 + 경기_성남시 + 경기_수원시 + 경기_시흥시 + 경기_안산시 + 경기_안성시 + 경기_안양시 + 경기_양주시 + 경기_양평군 + 경기_여주시 + 경기_연천군 + 경기_오산시 + 경기_용인시 + 경기_의왕시 + 경기_의정부시 + 경기_이천시 + 경기_파주시 + 경기_평택시 + 경기_포천시 + 경기_하남시 + 경기_화성시 + 서울_강남구 + 서울_강동구 + 서울_강북구 + 서울_강서구 + 서울_관악구 + 서울_광진구 + 서울_구로구 + 서울_노원구 + 서울_도봉구 + 서울_동대문구 + 서울_동작구 + 서울_서대문구 + 서울_서초구 + 서울_성동구 + 서울_성북구 + 서울_송파구 + 서울_양천구 + 서울_영등포구 + 서울_은평구 + 서울_종로구 + 서울_중구 + 서울_중랑구 + 인천_강화군 + 인천_계양구 + 인천_남동구 + 인천_동구 + 인천_미추홀구 + 인천_부평구 + 인천_서구 + 인천_연수구 + 인천_중구']
#                 소매 + 음식 + 교육 + 유원지오락 + 총인구수

formula = '공급액_만원 ~ 세대수 + 최고층 + 최저층 + 총아파트동수 + 용적률 + 건폐율 + 세대평균_주차대수 + 공급면적 + 방수 + 욕실수 + 대형건설사_YES + 지하철역_거리_km + 병원수_1차 + 병원수_2차 + 병원수_3차 + 공원 + 대학 + 장례식장 + 보건의료 + 서울'
results = smf.ols(formula, apt).fit()

print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                 공급액_만원   R-squared:                       0.745
Model:                            OLS   Adj. R-squared:                  0.743
Method:                 Least Squares   F-statistic:                     410.5
Date:                Sat, 30 Sep 2023   Prob (F-statistic):               0.00
Time:                        23:25:39   Log-Likelihood:                -31287.
No. Observations:                2828   AIC:                         6.262e+04
Df Residuals:                    2807   BIC:                         6.274e+04
Df Model:                          20                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept    -3.9e+04   2673.736    -14.586      0.0

In [98]:
output_filename = 'regression_result.txt'
with open(output_filename, 'w') as f:
    f.write(results.summary().as_text()) 

## **7-2) 7차 회귀분석 VIF** 

In [99]:
# 회귀 모델에 사용할 독립 변수 선택
independent_vars = ['세대수','최고층','최저층','총아파트동수', '대형건설사_YES',
                    '용적률','건폐율','세대평균_주차대수','공급면적','방수','욕실수',
                    '지하철역_거리_km','병원수_1차','병원수_2차','병원수_3차','공원','대학',
                    '장례식장','보건의료', '서울']


# 회귀 모델 적합
X = sm.add_constant(apt[independent_vars])
y = apt['공급액_만원']
model = sm.OLS(y, X)
results = model.fit()

In [100]:
# GPT 코드 <VIF가 10 이상인 것만 뽑아내기>

high_vif_vars = []
high_vif_values = []
threshold = 10  #임계값 설정

for i in range(1, X.shape[1]):  # 첫 번째 컬럼은 상수항이므로 제외
    vif = variance_inflation_factor(X.values, i)
    if vif >= threshold:
        high_vif_vars.append(X.columns[i])
        high_vif_values.append(vif)

print("Variables with VIF >= {}:".format(threshold))
for var, vif in zip(high_vif_vars, high_vif_values):
    print("{}: {:.2f}".format(var, vif))

Variables with VIF >= 10:


## **8-1) 8차 회귀분석** 

In [101]:
# 8차 ols표 

import statsmodels.formula. api as smf

# 포함하지 않을 X = ['아파트명','법정동주소','지하철역','역사명','노선명_리스트','기초','광역기초', '임대세대수' 
#                 위도 + 경도 + 최대공급면적 + 최소공급면적 + 전용면적 + 전용율 + 입주예정연도
#                 서울 + 경기 + 인천 + 혼합식 + 계단식 + 복도식 +
#                 가평군 + 강남구 + 강동구 + 강북구 + 강서구 + 강화군 + 계양구 + 고양시 + 과천시 + 관악구 + 광명시 + 광주시 + 광진구 + 구로구 + 구리시 + 군포시 + 김포시 + 남동구 + 남양주시 + 노원구 + 도봉구 + 동구 + 동대문구 + 동두천시 + 동작구 + 미추홀구 + 부천시 + 부평구 + 서구 + 서대문구 + 서초구 + 성남시 + 성동구 + 성북구 + 송파구 + 수원시 + 시흥시 + 안산시 + 안성시 + 안양시 + 양주시 + 양천구 + 양평군 + 여주시 + 연수구 + 연천군 + 영등포구 + 오산시 + 용인시 + 은평구 + 의왕시 + 의정부시 + 이천시 + 종로구 + 중구 + 중랑구 + 파주시 + 평택시 + 포천시 + 하남시 + 화성시
#                 _419민주묘지역 + 가양역 + 가좌역 + 가평역 + 갈산역 + 강일역 + 거여역 + 건대입구역 + 걸포북변역 + 경기광주역 + 경인교대입구역 + 경전철의정부역 + 계양역 + 고덕역 + 고속터미널역 + 고진역 + 고촌역 + 광교역 + 광명사거리역 + 광명역 + 구래역 + 구리역 + 구의역 + 국제업무지구역 + 군포역 + 금곡역 + 금릉역 + 금정역 + 길동역 + 까치울역 + 남구로역 + 남부터미널역 + 남한산성입구역 + 녹천역 + 능곡역 + 달미역 + 당고개역 + 덕계역 + 덕소역 + 덕정역 + 도농역 + 도심역 + 도원역 + 독정역 + 돌곶이역 + 동두천중앙역 + 동수역 + 동오역 + 동인천역 + 동천역 + 동춘역 + 둔전역 + 둔촌오륜역 + 등촌역 + 디지털미디어시티역 + 마곡역 + 마석역 + 마전역 + 마천역 + 망우역 + 망포역 + 매교역 + 매탄권선역 + 면목역 + 명지대역 + 명학역 + 모란역 + 문산역 + 문학경기장역 + 미아역 + 박촌역 + 발산역 + 백운역 + 범계역 + 범골역 + 별내역 + 병점역 + 보평역 + 봉천역 + 부개역 + 부발역 + 부천역 + 부평역 + 산성역 + 삼가역 + 삼동역 + 삼성역 + 삼양사거리역 + 상갈역 + 상도역 + 상동역 + 상일동역 + 상현역 + 서동탄역 + 서울대벤처타운역 + 서울대입구역 + 서정리역 + 석남역 + 석바위시장역 + 선부역 + 성환역 + 세류역 + 세마역 + 센트럴파크역 + 소새울역 + 소요산역 + 송도달빛축제공원역 + 송정역 + 송탄역 + 수색역 + 숭의역 + 시민공원역 + 시흥시청역 + 신길역 + 신길온천역 + 신답역 + 신반포역 + 신설동역 + 신원역 + 신정네거리역 + 신천역 + 쌍문역 + 아시아드경기장역 + 안산역 + 안암역 + 안양역 + 야당역 + 야목역 + 양주역 + 양천향교역 + 양촌역 + 양평역 + 어정역 + 어천역 + 여주역 + 역곡역 + 연수역 + 영등포구청역 + 영종역 + 예술회관역 + 오목천역 + 오빈역 + 오산대역 + 오산역 + 완정역 + 왕길역 + 외대앞역 + 용마산역 + 용문역 + 운동장송담대역 + 운서역 + 운정역 + 원당역 + 을지로4가역 + 응암역 + 의정부시청역 + 의정부역 + 의정부중앙역 + 이천역 + 인덕원역 + 인천대입구역 + 인천시청역 + 인하대역 + 일원역 + 작전역 + 장한평역 + 정부과천청사역 + 정왕역 + 제물포역 + 주안역 + 죽전역 + 중동역 + 중앙역 + 중화역 + 지식정보단지역 + 지제역 + 지행역 + 진위역 + 진접역 + 창신역 + 천왕역 + 천호역 + 철산역 + 청라국제도시역 + 청량리역 + 청명역 + 초월역 + 탑석역 + 테크노파크역 + 퇴계원역 + 파주역 + 평촌역 + 평택역 + 하남풍산역 + 행신역 + 화곡역 + 화서역 + 화전역 + 흑석역 + 흥선역
#                 경기_가평군 + 경기_고양시 + 경기_과천시 + 경기_광명시 + 경기_광주시 + 경기_구리시 + 경기_군포시 + 경기_김포시 + 경기_남양주시 + 경기_동두천시 + 경기_부천시 + 경기_성남시 + 경기_수원시 + 경기_시흥시 + 경기_안산시 + 경기_안성시 + 경기_안양시 + 경기_양주시 + 경기_양평군 + 경기_여주시 + 경기_연천군 + 경기_오산시 + 경기_용인시 + 경기_의왕시 + 경기_의정부시 + 경기_이천시 + 경기_파주시 + 경기_평택시 + 경기_포천시 + 경기_하남시 + 경기_화성시 + 서울_강남구 + 서울_강동구 + 서울_강북구 + 서울_강서구 + 서울_관악구 + 서울_광진구 + 서울_구로구 + 서울_노원구 + 서울_도봉구 + 서울_동대문구 + 서울_동작구 + 서울_서대문구 + 서울_서초구 + 서울_성동구 + 서울_성북구 + 서울_송파구 + 서울_양천구 + 서울_영등포구 + 서울_은평구 + 서울_종로구 + 서울_중구 + 서울_중랑구 + 인천_강화군 + 인천_계양구 + 인천_남동구 + 인천_동구 + 인천_미추홀구 + 인천_부평구 + 인천_서구 + 인천_연수구 + 인천_중구']
#                 소매 + 음식 + 교육 + 유원지오락 + 총인구수 + 병원수_1차 + 병원수_2차 + 공원 + 방수 + 욕실수

formula = '공급액_만원 ~ 세대수 + 최고층 + 최저층 + 총아파트동수 + 용적률 + 건폐율 + 세대평균_주차대수 + 공급면적 + 대형건설사_YES + 지하철역_거리_km  + 병원수_3차 + 대학 + 장례식장 + 보건의료 + 서울'
results = smf.ols(formula, apt).fit()

print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                 공급액_만원   R-squared:                       0.745
Model:                            OLS   Adj. R-squared:                  0.744
Method:                 Least Squares   F-statistic:                     547.4
Date:                Sat, 30 Sep 2023   Prob (F-statistic):               0.00
Time:                        23:25:47   Log-Likelihood:                -31288.
No. Observations:                2828   AIC:                         6.261e+04
Df Residuals:                    2812   BIC:                         6.270e+04
Df Model:                          15                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept  -4.018e+04   1917.536    -20.954      0.0

In [102]:
output_filename = 'regression_result.txt'
with open(output_filename, 'w') as f:
    f.write(results.summary().as_text()) 

## **8-2) 8차 회귀분석 VIF** 

In [103]:
# 회귀 모델에 사용할 독립 변수 선택
independent_vars = ['세대수','최고층','최저층','총아파트동수', '대형건설사_YES',
                    '용적률','건폐율','세대평균_주차대수','공급면적',
                    '지하철역_거리_km','병원수_3차','대학',
                    '장례식장','보건의료', '서울']


# 회귀 모델 적합
X = sm.add_constant(apt[independent_vars])
y = apt['공급액_만원']
model = sm.OLS(y, X)
results = model.fit()

In [104]:
# GPT 코드 <VIF가 10 이상인 것만 뽑아내기>

high_vif_vars = []
high_vif_values = []
threshold = 10  #임계값 설정

for i in range(1, X.shape[1]):  # 첫 번째 컬럼은 상수항이므로 제외
    vif = variance_inflation_factor(X.values, i)
    if vif >= threshold:
        high_vif_vars.append(X.columns[i])
        high_vif_values.append(vif)

print("Variables with VIF >= {}:".format(threshold))
for var, vif in zip(high_vif_vars, high_vif_values):
    print("{}: {:.2f}".format(var, vif))

Variables with VIF >= 10:


## **9-1) 9차 회귀분석** 

In [105]:
# 9차 ols표 

import statsmodels.formula. api as smf

# 포함하지 않을 X = ['아파트명','법정동주소','지하철역','역사명','노선명_리스트','기초','광역기초', '임대세대수' 
#                 위도 + 경도 + 최대공급면적 + 최소공급면적 + 공급면적 + 전용율 + 입주예정연도
#                 경기 + 인천 + 혼합식 + 계단식 + 복도식 +
#                 가평군 + 강남구 + 강동구 + 강북구 + 강서구 + 강화군 + 계양구 + 고양시 + 과천시 + 관악구 + 광명시 + 광주시 + 광진구 + 구로구 + 구리시 + 군포시 + 김포시 + 남동구 + 남양주시 + 노원구 + 도봉구 + 동구 + 동대문구 + 동두천시 + 동작구 + 미추홀구 + 부천시 + 부평구 + 서구 + 서대문구 + 서초구 + 성남시 + 성동구 + 성북구 + 송파구 + 수원시 + 시흥시 + 안산시 + 안성시 + 안양시 + 양주시 + 양천구 + 양평군 + 여주시 + 연수구 + 연천군 + 영등포구 + 오산시 + 용인시 + 은평구 + 의왕시 + 의정부시 + 이천시 + 종로구 + 중구 + 중랑구 + 파주시 + 평택시 + 포천시 + 하남시 + 화성시
#                 _419민주묘지역 + 가양역 + 가좌역 + 가평역 + 갈산역 + 강일역 + 거여역 + 건대입구역 + 걸포북변역 + 경기광주역 + 경인교대입구역 + 경전철의정부역 + 계양역 + 고덕역 + 고속터미널역 + 고진역 + 고촌역 + 광교역 + 광명사거리역 + 광명역 + 구래역 + 구리역 + 구의역 + 국제업무지구역 + 군포역 + 금곡역 + 금릉역 + 금정역 + 길동역 + 까치울역 + 남구로역 + 남부터미널역 + 남한산성입구역 + 녹천역 + 능곡역 + 달미역 + 당고개역 + 덕계역 + 덕소역 + 덕정역 + 도농역 + 도심역 + 도원역 + 독정역 + 돌곶이역 + 동두천중앙역 + 동수역 + 동오역 + 동인천역 + 동천역 + 동춘역 + 둔전역 + 둔촌오륜역 + 등촌역 + 디지털미디어시티역 + 마곡역 + 마석역 + 마전역 + 마천역 + 망우역 + 망포역 + 매교역 + 매탄권선역 + 면목역 + 명지대역 + 명학역 + 모란역 + 문산역 + 문학경기장역 + 미아역 + 박촌역 + 발산역 + 백운역 + 범계역 + 범골역 + 별내역 + 병점역 + 보평역 + 봉천역 + 부개역 + 부발역 + 부천역 + 부평역 + 산성역 + 삼가역 + 삼동역 + 삼성역 + 삼양사거리역 + 상갈역 + 상도역 + 상동역 + 상일동역 + 상현역 + 서동탄역 + 서울대벤처타운역 + 서울대입구역 + 서정리역 + 석남역 + 석바위시장역 + 선부역 + 성환역 + 세류역 + 세마역 + 센트럴파크역 + 소새울역 + 소요산역 + 송도달빛축제공원역 + 송정역 + 송탄역 + 수색역 + 숭의역 + 시민공원역 + 시흥시청역 + 신길역 + 신길온천역 + 신답역 + 신반포역 + 신설동역 + 신원역 + 신정네거리역 + 신천역 + 쌍문역 + 아시아드경기장역 + 안산역 + 안암역 + 안양역 + 야당역 + 야목역 + 양주역 + 양천향교역 + 양촌역 + 양평역 + 어정역 + 어천역 + 여주역 + 역곡역 + 연수역 + 영등포구청역 + 영종역 + 예술회관역 + 오목천역 + 오빈역 + 오산대역 + 오산역 + 완정역 + 왕길역 + 외대앞역 + 용마산역 + 용문역 + 운동장송담대역 + 운서역 + 운정역 + 원당역 + 을지로4가역 + 응암역 + 의정부시청역 + 의정부역 + 의정부중앙역 + 이천역 + 인덕원역 + 인천대입구역 + 인천시청역 + 인하대역 + 일원역 + 작전역 + 장한평역 + 정부과천청사역 + 정왕역 + 제물포역 + 주안역 + 죽전역 + 중동역 + 중앙역 + 중화역 + 지식정보단지역 + 지제역 + 지행역 + 진위역 + 진접역 + 창신역 + 천왕역 + 천호역 + 철산역 + 청라국제도시역 + 청량리역 + 청명역 + 초월역 + 탑석역 + 테크노파크역 + 퇴계원역 + 파주역 + 평촌역 + 평택역 + 하남풍산역 + 행신역 + 화곡역 + 화서역 + 화전역 + 흑석역 + 흥선역
#                 경기_가평군 + 경기_고양시 + 경기_과천시 + 경기_광명시 + 경기_광주시 + 경기_구리시 + 경기_군포시 + 경기_김포시 + 경기_남양주시 + 경기_동두천시 + 경기_부천시 + 경기_성남시 + 경기_수원시 + 경기_시흥시 + 경기_안산시 + 경기_안성시 + 경기_안양시 + 경기_양주시 + 경기_양평군 + 경기_여주시 + 경기_연천군 + 경기_오산시 + 경기_용인시 + 경기_의왕시 + 경기_의정부시 + 경기_이천시 + 경기_파주시 + 경기_평택시 + 경기_포천시 + 경기_하남시 + 경기_화성시 + 서울_강남구 + 서울_강동구 + 서울_강북구 + 서울_강서구 + 서울_관악구 + 서울_광진구 + 서울_구로구 + 서울_노원구 + 서울_도봉구 + 서울_동대문구 + 서울_동작구 + 서울_서대문구 + 서울_서초구 + 서울_성동구 + 서울_성북구 + 서울_송파구 + 서울_양천구 + 서울_영등포구 + 서울_은평구 + 서울_종로구 + 서울_중구 + 서울_중랑구 + 인천_강화군 + 인천_계양구 + 인천_남동구 + 인천_동구 + 인천_미추홀구 + 인천_부평구 + 인천_서구 + 인천_연수구 + 인천_중구']
#                 소매 + 음식 + 교육 + 유원지오락 + 총인구수 + 병원수_1차 + 병원수_2차 + 공원 + 방수 + 욕실수

formula = '공급액_만원 ~ 세대수 + 최고층 + 최저층 + 총아파트동수 + 용적률 + 건폐율 + 세대평균_주차대수 + 전용면적 + 대형건설사_YES + 지하철역_거리_km  + 병원수_3차 + 대학 + 장례식장 + 보건의료 + 서울'
results = smf.ols(formula, apt).fit()

print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                 공급액_만원   R-squared:                       0.734
Model:                            OLS   Adj. R-squared:                  0.733
Method:                 Least Squares   F-statistic:                     518.4
Date:                Sat, 30 Sep 2023   Prob (F-statistic):               0.00
Time:                        23:25:52   Log-Likelihood:                -31345.
No. Observations:                2828   AIC:                         6.272e+04
Df Residuals:                    2812   BIC:                         6.282e+04
Df Model:                          15                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept  -3.897e+04   1953.265    -19.953      0.0

## **9-2) 9차 회귀분석 VIF** 

In [106]:
# 회귀 모델에 사용할 독립 변수 선택
independent_vars = ['세대수','최고층','최저층','총아파트동수', '대형건설사_YES',
                    '용적률','건폐율','세대평균_주차대수','전용면적',
                    '지하철역_거리_km','병원수_3차','대학',
                    '장례식장','보건의료', '서울']


# 회귀 모델 적합
X = sm.add_constant(apt[independent_vars])
y = apt['공급액_만원']
model = sm.OLS(y, X)
results = model.fit()

In [107]:
# GPT 코드 <VIF가 10 이상인 것만 뽑아내기>

high_vif_vars = []
high_vif_values = []
threshold = 10  #임계값 설정

for i in range(1, X.shape[1]):  # 첫 번째 컬럼은 상수항이므로 제외
    vif = variance_inflation_factor(X.values, i)
    if vif >= threshold:
        high_vif_vars.append(X.columns[i])
        high_vif_values.append(vif)

print("Variables with VIF >= {}:".format(threshold))
for var, vif in zip(high_vif_vars, high_vif_values):
    print("{}: {:.2f}".format(var, vif))

Variables with VIF >= 10:


## **10-1) 10차 회귀분석** 

In [108]:
# 10차 ols표 

import statsmodels.formula. api as smf

# 포함하지 않을 X = ['아파트명','법정동주소','지하철역','역사명','노선명_리스트','기초','광역기초', '임대세대수' 
#                 위도 + 경도 + 최대공급면적 + 최소공급면적 + 공급면적 + 전용율 + 입주예정연도
#                 서울 + 경기 + 인천 + 혼합식 + 계단식 + 복도식 +
#                 가평군 + 강남구 + 강동구 + 강북구 + 강서구 + 강화군 + 계양구 + 고양시 + 과천시 + 관악구 + 광명시 + 광주시 + 광진구 + 구로구 + 구리시 + 군포시 + 김포시 + 남동구 + 남양주시 + 노원구 + 도봉구 + 동구 + 동대문구 + 동두천시 + 동작구 + 미추홀구 + 부천시 + 부평구 + 서구 + 서대문구 + 서초구 + 성남시 + 성동구 + 성북구 + 송파구 + 수원시 + 시흥시 + 안산시 + 안성시 + 안양시 + 양주시 + 양천구 + 양평군 + 여주시 + 연수구 + 연천군 + 영등포구 + 오산시 + 용인시 + 은평구 + 의왕시 + 의정부시 + 이천시 + 종로구 + 중구 + 중랑구 + 파주시 + 평택시 + 포천시 + 하남시 + 화성시
#                 _419민주묘지역 + 가양역 + 가좌역 + 가평역 + 갈산역 + 강일역 + 거여역 + 건대입구역 + 걸포북변역 + 경기광주역 + 경인교대입구역 + 경전철의정부역 + 계양역 + 고덕역 + 고속터미널역 + 고진역 + 고촌역 + 광교역 + 광명사거리역 + 광명역 + 구래역 + 구리역 + 구의역 + 국제업무지구역 + 군포역 + 금곡역 + 금릉역 + 금정역 + 길동역 + 까치울역 + 남구로역 + 남부터미널역 + 남한산성입구역 + 녹천역 + 능곡역 + 달미역 + 당고개역 + 덕계역 + 덕소역 + 덕정역 + 도농역 + 도심역 + 도원역 + 독정역 + 돌곶이역 + 동두천중앙역 + 동수역 + 동오역 + 동인천역 + 동천역 + 동춘역 + 둔전역 + 둔촌오륜역 + 등촌역 + 디지털미디어시티역 + 마곡역 + 마석역 + 마전역 + 마천역 + 망우역 + 망포역 + 매교역 + 매탄권선역 + 면목역 + 명지대역 + 명학역 + 모란역 + 문산역 + 문학경기장역 + 미아역 + 박촌역 + 발산역 + 백운역 + 범계역 + 범골역 + 별내역 + 병점역 + 보평역 + 봉천역 + 부개역 + 부발역 + 부천역 + 부평역 + 산성역 + 삼가역 + 삼동역 + 삼성역 + 삼양사거리역 + 상갈역 + 상도역 + 상동역 + 상일동역 + 상현역 + 서동탄역 + 서울대벤처타운역 + 서울대입구역 + 서정리역 + 석남역 + 석바위시장역 + 선부역 + 성환역 + 세류역 + 세마역 + 센트럴파크역 + 소새울역 + 소요산역 + 송도달빛축제공원역 + 송정역 + 송탄역 + 수색역 + 숭의역 + 시민공원역 + 시흥시청역 + 신길역 + 신길온천역 + 신답역 + 신반포역 + 신설동역 + 신원역 + 신정네거리역 + 신천역 + 쌍문역 + 아시아드경기장역 + 안산역 + 안암역 + 안양역 + 야당역 + 야목역 + 양주역 + 양천향교역 + 양촌역 + 양평역 + 어정역 + 어천역 + 여주역 + 역곡역 + 연수역 + 영등포구청역 + 영종역 + 예술회관역 + 오목천역 + 오빈역 + 오산대역 + 오산역 + 완정역 + 왕길역 + 외대앞역 + 용마산역 + 용문역 + 운동장송담대역 + 운서역 + 운정역 + 원당역 + 을지로4가역 + 응암역 + 의정부시청역 + 의정부역 + 의정부중앙역 + 이천역 + 인덕원역 + 인천대입구역 + 인천시청역 + 인하대역 + 일원역 + 작전역 + 장한평역 + 정부과천청사역 + 정왕역 + 제물포역 + 주안역 + 죽전역 + 중동역 + 중앙역 + 중화역 + 지식정보단지역 + 지제역 + 지행역 + 진위역 + 진접역 + 창신역 + 천왕역 + 천호역 + 철산역 + 청라국제도시역 + 청량리역 + 청명역 + 초월역 + 탑석역 + 테크노파크역 + 퇴계원역 + 파주역 + 평촌역 + 평택역 + 하남풍산역 + 행신역 + 화곡역 + 화서역 + 화전역 + 흑석역 + 흥선역
#                 경기_가평군 + 경기_고양시 + 경기_과천시 + 경기_광명시 + 경기_광주시 + 경기_구리시 + 경기_군포시 + 경기_김포시 + 경기_남양주시 + 경기_동두천시 + 경기_부천시 + 경기_성남시 + 경기_수원시 + 경기_시흥시 + 경기_안산시 + 경기_안성시 + 경기_안양시 + 경기_양주시 + 경기_양평군 + 경기_여주시 + 경기_연천군 + 경기_오산시 + 경기_용인시 + 경기_의왕시 + 경기_의정부시 + 경기_이천시 + 경기_파주시 + 경기_평택시 + 경기_포천시 + 경기_하남시 + 경기_화성시 + 인천_강화군 + 인천_계양구 + 인천_남동구 + 인천_동구 + 인천_미추홀구 + 인천_부평구 + 인천_서구 + 인천_연수구 + 인천_중구']
#                 소매 + 음식 + 교육 + 유원지오락 + 총인구수 + 병원수_1차 + 병원수_2차 + 공원 + 방수 + 욕실수

formula = '공급액_만원 ~ 세대수 + 최고층 + 최저층 + 총아파트동수 + 용적률 + 건폐율 + 세대평균_주차대수 + 전용면적 + 대형건설사_YES + 지하철역_거리_km + 병원수_3차 + 대학 + 장례식장 + 보건의료 +  서울_강남구 + 서울_강동구 + 서울_강북구 + 서울_강서구 + 서울_관악구 + 서울_광진구 + 서울_구로구 + 서울_노원구 + 서울_도봉구 + 서울_동대문구 + 서울_동작구 + 서울_서대문구 + 서울_서초구 + 서울_성동구 + 서울_성북구 + 서울_송파구 + 서울_양천구 + 서울_영등포구 + 서울_은평구 + 서울_종로구 + 서울_중구 + 서울_중랑구 '
results = smf.ols(formula, apt).fit()

print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                 공급액_만원   R-squared:                       0.774
Model:                            OLS   Adj. R-squared:                  0.771
Method:                 Least Squares   F-statistic:                     265.7
Date:                Sat, 30 Sep 2023   Prob (F-statistic):               0.00
Time:                        23:25:53   Log-Likelihood:                -31116.
No. Observations:                2828   AIC:                         6.231e+04
Df Residuals:                    2791   BIC:                         6.253e+04
Df Model:                          36                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept  -3.781e+04   1940.803    -19.483      0.0

## **10-2) 10차 회귀분석 VIF** 

In [109]:
# 회귀 모델에 사용할 독립 변수 선택
independent_vars = ['세대수','최고층','최저층','총아파트동수', '대형건설사_YES',
                    '용적률','건폐율','세대평균_주차대수','전용면적',
                    '지하철역_거리_km','병원수_3차','대학','장례식장','보건의료', 
                    '서울_강남구','서울_강동구','서울_강북구','서울_강서구','서울_관악구','서울_광진구','서울_구로구',
                    '서울_노원구','서울_도봉구','서울_동대문구','서울_동작구','서울_서대문구','서울_서초구','서울_성동구','서울_성북구','서울_송파구',
                    '서울_양천구','서울_영등포구','서울_은평구','서울_종로구','서울_중구','서울_중랑구']


# 회귀 모델 적합
X = sm.add_constant(apt[independent_vars])
y = apt['공급액_만원']
model = sm.OLS(y, X)
results = model.fit()

In [110]:
# GPT 코드 <VIF가 10 이상인 것만 뽑아내기>

high_vif_vars = []
high_vif_values = []
threshold = 10  #임계값 설정

for i in range(1, X.shape[1]):  # 첫 번째 컬럼은 상수항이므로 제외
    vif = variance_inflation_factor(X.values, i)
    if vif >= threshold:
        high_vif_vars.append(X.columns[i])
        high_vif_values.append(vif)

print("Variables with VIF >= {}:".format(threshold))
for var, vif in zip(high_vif_vars, high_vif_values):
    print("{}: {:.2f}".format(var, vif))

Variables with VIF >= 10:
세대수: 10.45


## **11-1) 11차 회귀분석** 

In [111]:
# 11차 ols표 

import statsmodels.formula. api as smf

# 포함하지 않을 X = ['아파트명','법정동주소','지하철역','역사명','노선명_리스트','기초','광역기초', '임대세대수' 
#                 위도 + 경도 + 최대공급면적 + 최소공급면적 + 전용율 + 입주예정연도
#                 서울 + 경기 + 인천 + 혼합식 + 계단식 + 복도식 +
#                 가평군 + 강남구 + 강동구 + 강북구 + 강서구 + 강화군 + 계양구 + 고양시 + 과천시 + 관악구 + 광명시 + 광주시 + 광진구 + 구로구 + 구리시 + 군포시 + 김포시 + 남동구 + 남양주시 + 노원구 + 도봉구 + 동구 + 동대문구 + 동두천시 + 동작구 + 미추홀구 + 부천시 + 부평구 + 서구 + 서대문구 + 서초구 + 성남시 + 성동구 + 성북구 + 송파구 + 수원시 + 시흥시 + 안산시 + 안성시 + 안양시 + 양주시 + 양천구 + 양평군 + 여주시 + 연수구 + 연천군 + 영등포구 + 오산시 + 용인시 + 은평구 + 의왕시 + 의정부시 + 이천시 + 종로구 + 중구 + 중랑구 + 파주시 + 평택시 + 포천시 + 하남시 + 화성시
#                 _419민주묘지역 + 가양역 + 가좌역 + 가평역 + 갈산역 + 강일역 + 거여역 + 건대입구역 + 걸포북변역 + 경기광주역 + 경인교대입구역 + 경전철의정부역 + 계양역 + 고덕역 + 고속터미널역 + 고진역 + 고촌역 + 광교역 + 광명사거리역 + 광명역 + 구래역 + 구리역 + 구의역 + 국제업무지구역 + 군포역 + 금곡역 + 금릉역 + 금정역 + 길동역 + 까치울역 + 남구로역 + 남부터미널역 + 남한산성입구역 + 녹천역 + 능곡역 + 달미역 + 당고개역 + 덕계역 + 덕소역 + 덕정역 + 도농역 + 도심역 + 도원역 + 독정역 + 돌곶이역 + 동두천중앙역 + 동수역 + 동오역 + 동인천역 + 동천역 + 동춘역 + 둔전역 + 둔촌오륜역 + 등촌역 + 디지털미디어시티역 + 마곡역 + 마석역 + 마전역 + 마천역 + 망우역 + 망포역 + 매교역 + 매탄권선역 + 면목역 + 명지대역 + 명학역 + 모란역 + 문산역 + 문학경기장역 + 미아역 + 박촌역 + 발산역 + 백운역 + 범계역 + 범골역 + 별내역 + 병점역 + 보평역 + 봉천역 + 부개역 + 부발역 + 부천역 + 부평역 + 산성역 + 삼가역 + 삼동역 + 삼성역 + 삼양사거리역 + 상갈역 + 상도역 + 상동역 + 상일동역 + 상현역 + 서동탄역 + 서울대벤처타운역 + 서울대입구역 + 서정리역 + 석남역 + 석바위시장역 + 선부역 + 성환역 + 세류역 + 세마역 + 센트럴파크역 + 소새울역 + 소요산역 + 송도달빛축제공원역 + 송정역 + 송탄역 + 수색역 + 숭의역 + 시민공원역 + 시흥시청역 + 신길역 + 신길온천역 + 신답역 + 신반포역 + 신설동역 + 신원역 + 신정네거리역 + 신천역 + 쌍문역 + 아시아드경기장역 + 안산역 + 안암역 + 안양역 + 야당역 + 야목역 + 양주역 + 양천향교역 + 양촌역 + 양평역 + 어정역 + 어천역 + 여주역 + 역곡역 + 연수역 + 영등포구청역 + 영종역 + 예술회관역 + 오목천역 + 오빈역 + 오산대역 + 오산역 + 완정역 + 왕길역 + 외대앞역 + 용마산역 + 용문역 + 운동장송담대역 + 운서역 + 운정역 + 원당역 + 을지로4가역 + 응암역 + 의정부시청역 + 의정부역 + 의정부중앙역 + 이천역 + 인덕원역 + 인천대입구역 + 인천시청역 + 인하대역 + 일원역 + 작전역 + 장한평역 + 정부과천청사역 + 정왕역 + 제물포역 + 주안역 + 죽전역 + 중동역 + 중앙역 + 중화역 + 지식정보단지역 + 지제역 + 지행역 + 진위역 + 진접역 + 창신역 + 천왕역 + 천호역 + 철산역 + 청라국제도시역 + 청량리역 + 청명역 + 초월역 + 탑석역 + 테크노파크역 + 퇴계원역 + 파주역 + 평촌역 + 평택역 + 하남풍산역 + 행신역 + 화곡역 + 화서역 + 화전역 + 흑석역 + 흥선역
#                 경기_가평군 + 경기_고양시 + 경기_과천시 + 경기_광명시 + 경기_광주시 + 경기_구리시 + 경기_군포시 + 경기_김포시 + 경기_남양주시 + 경기_동두천시 + 경기_부천시 + 경기_성남시 + 경기_수원시 + 경기_시흥시 + 경기_안산시 + 경기_안성시 + 경기_안양시 + 경기_양주시 + 경기_양평군 + 경기_여주시 + 경기_연천군 + 경기_오산시 + 경기_용인시 + 경기_의왕시 + 경기_의정부시 + 경기_이천시 + 경기_파주시 + 경기_평택시 + 경기_포천시 + 경기_하남시 + 경기_화성시 + 인천_강화군 + 인천_계양구 + 인천_남동구 + 인천_동구 + 인천_미추홀구 + 인천_부평구 + 인천_서구 + 인천_연수구 + 인천_중구']
#                 서울_노원구  + 서울_송파구
#                 소매 + 음식 + 교육 + 유원지오락 + 총인구수 + 병원수_1차 + 병원수_2차 + 공원 + 방수 + 욕실수

formula = '공급액_만원 ~ 세대수 + 최고층 + 최저층 + 총아파트동수 + 용적률 + 건폐율 + 세대평균_주차대수 + 공급면적 + 전용면적 + 대형건설사_YES + 지하철역_거리_km + 병원수_3차 + 대학 + 장례식장 + 보건의료 +  서울_강남구 + 서울_강동구 + 서울_강북구 + 서울_강서구 + 서울_관악구 + 서울_광진구 + 서울_구로구 + 서울_도봉구 + 서울_동대문구 + 서울_동작구 + 서울_서대문구 + 서울_서초구 + 서울_성동구 + 서울_성북구 + 서울_양천구 + 서울_영등포구 + 서울_은평구 + 서울_종로구 + 서울_중구 + 서울_중랑구 '
results = smf.ols(formula, apt).fit()

print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                 공급액_만원   R-squared:                       0.788
Model:                            OLS   Adj. R-squared:                  0.786
Method:                 Least Squares   F-statistic:                     297.4
Date:                Sat, 30 Sep 2023   Prob (F-statistic):               0.00
Time:                        23:25:55   Log-Likelihood:                -31023.
No. Observations:                2828   AIC:                         6.212e+04
Df Residuals:                    2792   BIC:                         6.233e+04
Df Model:                          35                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept  -3.893e+04   1876.051    -20.749      0.0

## **11-2) 11차 회귀분석 VIF** 

In [112]:
# 회귀 모델에 사용할 독립 변수 선택
independent_vars = ['세대수','최고층','최저층','총아파트동수', '대형건설사_YES',
                    '용적률','건폐율','세대평균_주차대수','공급면적', '전용면적',
                    '지하철역_거리_km','병원수_3차','대학','장례식장','보건의료', 
                    '서울_강남구','서울_강동구','서울_강북구','서울_강서구','서울_관악구','서울_광진구','서울_구로구',
                    '서울_도봉구','서울_동대문구','서울_동작구','서울_서대문구','서울_서초구','서울_성동구','서울_성북구',
                    '서울_양천구','서울_영등포구','서울_은평구','서울_종로구','서울_중구','서울_중랑구']


# 회귀 모델 적합
X = sm.add_constant(apt[independent_vars])
y = apt['공급액_만원']
model = sm.OLS(y, X)
results = model.fit()

In [113]:
# GPT 코드 <VIF가 10 이상인 것만 뽑아내기>

high_vif_vars = []
high_vif_values = []
threshold = 10  #임계값 설정

for i in range(1, X.shape[1]):  # 첫 번째 컬럼은 상수항이므로 제외
    vif = variance_inflation_factor(X.values, i)
    if vif >= threshold:
        high_vif_vars.append(X.columns[i])
        high_vif_values.append(vif)

print("Variables with VIF >= {}:".format(threshold))
for var, vif in zip(high_vif_vars, high_vif_values):
    print("{}: {:.2f}".format(var, vif))

Variables with VIF >= 10:
세대수: 10.42
공급면적: 75.50
전용면적: 74.60


#### 서울 구를 넣으니까 세대수 VIF가 높아짐.

## **12-1) 12차 회귀분석** 

In [114]:
# 12차 ols표 

import statsmodels.formula. api as smf

# 포함하지 않을 X = ['아파트명','법정동주소','지하철역','역사명','노선명_리스트','기초','광역기초', '임대세대수' 
#                 위도 + 경도 + 최대공급면적 + 최소공급면적 + 공급면적 + 전용율 + 입주예정연도
#                 서울 + 경기 + 인천 + 혼합식 + 계단식 + 복도식 +
#                 가평군 + 강남구 + 강동구 + 강북구 + 강서구 + 강화군 + 계양구 + 고양시 + 과천시 + 관악구 + 광명시 + 광주시 + 광진구 + 구로구 + 구리시 + 군포시 + 김포시 + 남동구 + 남양주시 + 노원구 + 도봉구 + 동구 + 동대문구 + 동두천시 + 동작구 + 미추홀구 + 부천시 + 부평구 + 서구 + 서대문구 + 서초구 + 성남시 + 성동구 + 성북구 + 송파구 + 수원시 + 시흥시 + 안산시 + 안성시 + 안양시 + 양주시 + 양천구 + 양평군 + 여주시 + 연수구 + 연천군 + 영등포구 + 오산시 + 용인시 + 은평구 + 의왕시 + 의정부시 + 이천시 + 종로구 + 중구 + 중랑구 + 파주시 + 평택시 + 포천시 + 하남시 + 화성시
#                 _419민주묘지역 + 가양역 + 가좌역 + 가평역 + 갈산역 + 강일역 + 거여역 + 건대입구역 + 걸포북변역 + 경기광주역 + 경인교대입구역 + 경전철의정부역 + 계양역 + 고덕역 + 고속터미널역 + 고진역 + 고촌역 + 광교역 + 광명사거리역 + 광명역 + 구래역 + 구리역 + 구의역 + 국제업무지구역 + 군포역 + 금곡역 + 금릉역 + 금정역 + 길동역 + 까치울역 + 남구로역 + 남부터미널역 + 남한산성입구역 + 녹천역 + 능곡역 + 달미역 + 당고개역 + 덕계역 + 덕소역 + 덕정역 + 도농역 + 도심역 + 도원역 + 독정역 + 돌곶이역 + 동두천중앙역 + 동수역 + 동오역 + 동인천역 + 동천역 + 동춘역 + 둔전역 + 둔촌오륜역 + 등촌역 + 디지털미디어시티역 + 마곡역 + 마석역 + 마전역 + 마천역 + 망우역 + 망포역 + 매교역 + 매탄권선역 + 면목역 + 명지대역 + 명학역 + 모란역 + 문산역 + 문학경기장역 + 미아역 + 박촌역 + 발산역 + 백운역 + 범계역 + 범골역 + 별내역 + 병점역 + 보평역 + 봉천역 + 부개역 + 부발역 + 부천역 + 부평역 + 산성역 + 삼가역 + 삼동역 + 삼성역 + 삼양사거리역 + 상갈역 + 상도역 + 상동역 + 상일동역 + 상현역 + 서동탄역 + 서울대벤처타운역 + 서울대입구역 + 서정리역 + 석남역 + 석바위시장역 + 선부역 + 성환역 + 세류역 + 세마역 + 센트럴파크역 + 소새울역 + 소요산역 + 송도달빛축제공원역 + 송정역 + 송탄역 + 수색역 + 숭의역 + 시민공원역 + 시흥시청역 + 신길역 + 신길온천역 + 신답역 + 신반포역 + 신설동역 + 신원역 + 신정네거리역 + 신천역 + 쌍문역 + 아시아드경기장역 + 안산역 + 안암역 + 안양역 + 야당역 + 야목역 + 양주역 + 양천향교역 + 양촌역 + 양평역 + 어정역 + 어천역 + 여주역 + 역곡역 + 연수역 + 영등포구청역 + 영종역 + 예술회관역 + 오목천역 + 오빈역 + 오산대역 + 오산역 + 완정역 + 왕길역 + 외대앞역 + 용마산역 + 용문역 + 운동장송담대역 + 운서역 + 운정역 + 원당역 + 을지로4가역 + 응암역 + 의정부시청역 + 의정부역 + 의정부중앙역 + 이천역 + 인덕원역 + 인천대입구역 + 인천시청역 + 인하대역 + 일원역 + 작전역 + 장한평역 + 정부과천청사역 + 정왕역 + 제물포역 + 주안역 + 죽전역 + 중동역 + 중앙역 + 중화역 + 지식정보단지역 + 지제역 + 지행역 + 진위역 + 진접역 + 창신역 + 천왕역 + 천호역 + 철산역 + 청라국제도시역 + 청량리역 + 청명역 + 초월역 + 탑석역 + 테크노파크역 + 퇴계원역 + 파주역 + 평촌역 + 평택역 + 하남풍산역 + 행신역 + 화곡역 + 화서역 + 화전역 + 흑석역 + 흥선역
#                 경기_가평군 + 경기_고양시 + 경기_과천시 + 경기_광명시 + 경기_광주시 + 경기_구리시 + 경기_군포시 + 경기_김포시 + 경기_남양주시 + 경기_동두천시 + 경기_부천시 + 경기_성남시 + 경기_수원시 + 경기_시흥시 + 경기_안산시 + 경기_안성시 + 경기_안양시 + 경기_양주시 + 경기_양평군 + 경기_여주시 + 경기_연천군 + 경기_오산시 + 경기_용인시 + 경기_의왕시 + 경기_의정부시 + 경기_이천시 + 경기_파주시 + 경기_평택시 + 경기_포천시 + 경기_하남시 + 경기_화성시 + 인천_강화군 + 인천_계양구 + 인천_남동구 + 인천_동구 + 인천_미추홀구 + 인천_부평구 + 인천_서구 + 인천_연수구 + 인천_중구']
#                 서울_노원구  + 서울_송파구
#                 소매 + 음식 + 교육 + 유원지오락 + 총인구수 + 병원수_1차 + 병원수_2차 + 공원 + 방수 + 욕실수 + 총아파트동수

formula = '공급액_만원 ~ 세대수 + 최고층 + 최저층 + 용적률 + 건폐율 + 세대평균_주차대수 + 전용면적 + 대형건설사_YES + 지하철역_거리_km + 병원수_3차 + 대학 + 장례식장 + 보건의료 +  서울_강남구 + 서울_강동구 + 서울_강북구 + 서울_강서구 + 서울_관악구 + 서울_광진구 + 서울_구로구 + 서울_도봉구 + 서울_동대문구 + 서울_동작구 + 서울_서대문구 + 서울_서초구 + 서울_성동구 + 서울_성북구 + 서울_양천구 + 서울_영등포구 + 서울_은평구 + 서울_종로구 + 서울_중구 + 서울_중랑구 '
results = smf.ols(formula, apt).fit()

print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                 공급액_만원   R-squared:                       0.764
Model:                            OLS   Adj. R-squared:                  0.761
Method:                 Least Squares   F-statistic:                     274.1
Date:                Sat, 30 Sep 2023   Prob (F-statistic):               0.00
Time:                        23:25:56   Log-Likelihood:                -31178.
No. Observations:                2828   AIC:                         6.242e+04
Df Residuals:                    2794   BIC:                         6.263e+04
Df Model:                          33                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept  -4.251e+04   1931.177    -22.011      0.0

In [115]:
output_filename = 'regression_result.txt'
with open(output_filename, 'w') as f:
    f.write(results.summary().as_text()) 

## **12-2) 12차 회귀분석 VIF** 

In [116]:
# 회귀 모델에 사용할 독립 변수 선택
independent_vars = ['세대수','최고층','최저층', '대형건설사_YES',
                    '용적률','건폐율','세대평균_주차대수','전용면적',
                    '지하철역_거리_km','병원수_3차','대학','장례식장','보건의료', 
                    '서울_강남구','서울_강동구','서울_강북구','서울_강서구','서울_관악구','서울_광진구','서울_구로구',
                    '서울_도봉구','서울_동대문구','서울_동작구','서울_서대문구','서울_서초구','서울_성동구','서울_성북구',
                    '서울_양천구','서울_영등포구','서울_은평구','서울_종로구','서울_중구','서울_중랑구']


# 회귀 모델 적합
X = sm.add_constant(apt[independent_vars])
y = apt['공급액_만원']
model = sm.OLS(y, X)
results = model.fit()

In [117]:
# GPT 코드 <VIF가 10 이상인 것만 뽑아내기>

high_vif_vars = []
high_vif_values = []
threshold = 10  #임계값 설정

for i in range(1, X.shape[1]):  # 첫 번째 컬럼은 상수항이므로 제외
    vif = variance_inflation_factor(X.values, i)
    if vif >= threshold:
        high_vif_vars.append(X.columns[i])
        high_vif_values.append(vif)

print("Variables with VIF >= {}:".format(threshold))
for var, vif in zip(high_vif_vars, high_vif_values):
    print("{}: {:.2f}".format(var, vif))

Variables with VIF >= 10:


## **13-1) 13차 회귀분석** 

In [118]:
# 12차 ols표 

import statsmodels.formula. api as smf

# 포함하지 않을 X = ['아파트명','법정동주소','지하철역','역사명','노선명_리스트','기초','광역기초', '임대세대수' 
#                 위도 + 경도 + 최대공급면적 + 최소공급면적 + 공급면적 + 전용율 + 입주예정연도
#                 서울 + 경기 + 인천 + 혼합식 + 계단식 + 복도식 +
#                 가평군 + 강남구 + 강동구 + 강북구 + 강서구 + 강화군 + 계양구 + 고양시 + 과천시 + 관악구 + 광명시 + 광주시 + 광진구 + 구로구 + 구리시 + 군포시 + 김포시 + 남동구 + 남양주시 + 노원구 + 도봉구 + 동구 + 동대문구 + 동두천시 + 동작구 + 미추홀구 + 부천시 + 부평구 + 서구 + 서대문구 + 서초구 + 성남시 + 성동구 + 성북구 + 송파구 + 수원시 + 시흥시 + 안산시 + 안성시 + 안양시 + 양주시 + 양천구 + 양평군 + 여주시 + 연수구 + 연천군 + 영등포구 + 오산시 + 용인시 + 은평구 + 의왕시 + 의정부시 + 이천시 + 종로구 + 중구 + 중랑구 + 파주시 + 평택시 + 포천시 + 하남시 + 화성시
#                 _419민주묘지역 + 가양역 + 가좌역 + 가평역 + 갈산역 + 강일역 + 거여역 + 건대입구역 + 걸포북변역 + 경기광주역 + 경인교대입구역 + 경전철의정부역 + 계양역 + 고덕역 + 고속터미널역 + 고진역 + 고촌역 + 광교역 + 광명사거리역 + 광명역 + 구래역 + 구리역 + 구의역 + 국제업무지구역 + 군포역 + 금곡역 + 금릉역 + 금정역 + 길동역 + 까치울역 + 남구로역 + 남부터미널역 + 남한산성입구역 + 녹천역 + 능곡역 + 달미역 + 당고개역 + 덕계역 + 덕소역 + 덕정역 + 도농역 + 도심역 + 도원역 + 독정역 + 돌곶이역 + 동두천중앙역 + 동수역 + 동오역 + 동인천역 + 동천역 + 동춘역 + 둔전역 + 둔촌오륜역 + 등촌역 + 디지털미디어시티역 + 마곡역 + 마석역 + 마전역 + 마천역 + 망우역 + 망포역 + 매교역 + 매탄권선역 + 면목역 + 명지대역 + 명학역 + 모란역 + 문산역 + 문학경기장역 + 미아역 + 박촌역 + 발산역 + 백운역 + 범계역 + 범골역 + 별내역 + 병점역 + 보평역 + 봉천역 + 부개역 + 부발역 + 부천역 + 부평역 + 산성역 + 삼가역 + 삼동역 + 삼성역 + 삼양사거리역 + 상갈역 + 상도역 + 상동역 + 상일동역 + 상현역 + 서동탄역 + 서울대벤처타운역 + 서울대입구역 + 서정리역 + 석남역 + 석바위시장역 + 선부역 + 성환역 + 세류역 + 세마역 + 센트럴파크역 + 소새울역 + 소요산역 + 송도달빛축제공원역 + 송정역 + 송탄역 + 수색역 + 숭의역 + 시민공원역 + 시흥시청역 + 신길역 + 신길온천역 + 신답역 + 신반포역 + 신설동역 + 신원역 + 신정네거리역 + 신천역 + 쌍문역 + 아시아드경기장역 + 안산역 + 안암역 + 안양역 + 야당역 + 야목역 + 양주역 + 양천향교역 + 양촌역 + 양평역 + 어정역 + 어천역 + 여주역 + 역곡역 + 연수역 + 영등포구청역 + 영종역 + 예술회관역 + 오목천역 + 오빈역 + 오산대역 + 오산역 + 완정역 + 왕길역 + 외대앞역 + 용마산역 + 용문역 + 운동장송담대역 + 운서역 + 운정역 + 원당역 + 을지로4가역 + 응암역 + 의정부시청역 + 의정부역 + 의정부중앙역 + 이천역 + 인덕원역 + 인천대입구역 + 인천시청역 + 인하대역 + 일원역 + 작전역 + 장한평역 + 정부과천청사역 + 정왕역 + 제물포역 + 주안역 + 죽전역 + 중동역 + 중앙역 + 중화역 + 지식정보단지역 + 지제역 + 지행역 + 진위역 + 진접역 + 창신역 + 천왕역 + 천호역 + 철산역 + 청라국제도시역 + 청량리역 + 청명역 + 초월역 + 탑석역 + 테크노파크역 + 퇴계원역 + 파주역 + 평촌역 + 평택역 + 하남풍산역 + 행신역 + 화곡역 + 화서역 + 화전역 + 흑석역 + 흥선역
#                 경기_가평군 + 경기_고양시 + 경기_과천시 + 경기_광명시 + 경기_광주시 + 경기_구리시 + 경기_군포시 + 경기_김포시 + 경기_남양주시 + 경기_동두천시 + 경기_부천시 + 경기_성남시 + 경기_수원시 + 경기_시흥시 + 경기_안산시 + 경기_안성시 + 경기_안양시 + 경기_양주시 + 경기_양평군 + 경기_여주시 + 경기_연천군 + 경기_오산시 + 경기_용인시 + 경기_의왕시 + 경기_의정부시 + 경기_이천시 + 경기_파주시 + 경기_평택시 + 경기_포천시 + 경기_하남시 + 경기_화성시 + 인천_강화군 + 인천_계양구 + 인천_남동구 + 인천_동구 + 인천_미추홀구 + 인천_부평구 + 인천_서구 + 인천_연수구 + 인천_중구']
#                 서울_노원구  + 서울_송파구
#                 소매 + 음식 + 교육 + 유원지오락 + 총인구수 + 병원수_1차 + 병원수_2차 + 공원 + 방수 + 욕실수 + 총아파트동수 + 대학

formula = '공급액_만원 ~ 세대수 + 최고층 + 최저층 + 용적률 + 건폐율 + 세대평균_주차대수 + 전용면적 + 대형건설사_YES + 지하철역_거리_km + 병원수_3차 + 장례식장 + 보건의료 +  서울_강남구 + 서울_강동구 + 서울_강북구 + 서울_강서구 + 서울_관악구 + 서울_광진구 + 서울_구로구 + 서울_도봉구 + 서울_동대문구 + 서울_동작구 + 서울_서대문구 + 서울_서초구 + 서울_성동구 + 서울_성북구 + 서울_양천구 + 서울_영등포구 + 서울_은평구 + 서울_종로구 + 서울_중구 + 서울_중랑구 '
results = smf.ols(formula, apt).fit()

print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                 공급액_만원   R-squared:                       0.764
Model:                            OLS   Adj. R-squared:                  0.761
Method:                 Least Squares   F-statistic:                     282.6
Date:                Sat, 30 Sep 2023   Prob (F-statistic):               0.00
Time:                        23:25:58   Log-Likelihood:                -31179.
No. Observations:                2828   AIC:                         6.242e+04
Df Residuals:                    2795   BIC:                         6.262e+04
Df Model:                          32                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept  -4.251e+04   1931.159    -22.012      0.0

In [119]:
output_filename = 'regression_result.txt'
with open(output_filename, 'w') as f:
    f.write(results.summary().as_text()) 

## **13-2) 13차 회귀분석 VIF** 

In [120]:
# 회귀 모델에 사용할 독립 변수 선택
independent_vars = ['세대수','최고층','최저층', '대형건설사_YES',
                    '용적률','건폐율','세대평균_주차대수','전용면적',
                    '지하철역_거리_km','병원수_3차','장례식장','보건의료', 
                    '서울_강남구','서울_강동구','서울_강북구','서울_강서구','서울_관악구','서울_광진구','서울_구로구',
                    '서울_도봉구','서울_동대문구','서울_동작구','서울_서대문구','서울_서초구','서울_성동구','서울_성북구',
                    '서울_양천구','서울_영등포구','서울_은평구','서울_종로구','서울_중구','서울_중랑구']


# 회귀 모델 적합
X = sm.add_constant(apt[independent_vars])
y = apt['공급액_만원']
model = sm.OLS(y, X)
results = model.fit()

In [121]:
# GPT 코드 <VIF가 10 이상인 것만 뽑아내기>

high_vif_vars = []
high_vif_values = []
threshold = 10  #임계값 설정

for i in range(1, X.shape[1]):  # 첫 번째 컬럼은 상수항이므로 제외
    vif = variance_inflation_factor(X.values, i)
    if vif >= threshold:
        high_vif_vars.append(X.columns[i])
        high_vif_values.append(vif)

print("Variables with VIF >= {}:".format(threshold))
for var, vif in zip(high_vif_vars, high_vif_values):
    print("{}: {:.2f}".format(var, vif))

Variables with VIF >= 10:
